# WESAD FastGRNN

Adapted from Microsoft's notebooks, available at https://github.com/microsoft/EdgeML authored by Dennis et al.

In [1]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import os
import datetime as datetime
import pickle as pkl
from sklearn.model_selection import train_test_split
import pathlib
from os import mkdir

In [2]:
def loadData(dirname):
    x_train = np.load(dirname + '/' + 'x_train.npy')
    y_train = np.load(dirname + '/' + 'y_train.npy')
    x_test = np.load(dirname + '/' + 'x_test.npy')
    y_test = np.load(dirname + '/' + 'y_test.npy')
    x_val = np.load(dirname + '/' + 'x_val.npy')
    y_val = np.load(dirname + '/' + 'y_val.npy')
    return x_train, y_train, x_test, y_test, x_val, y_val
def makeEMIData(subinstanceLen, subinstanceStride, sourceDir, outDir):
    x_train, y_train, x_test, y_test, x_val, y_val = loadData(sourceDir)
    x, y = bagData(x_train, y_train, subinstanceLen, subinstanceStride)
    np.save(outDir + '/x_train.npy', x)
    np.save(outDir + '/y_train.npy', y)
    print('Num train %d' % len(x))
    x, y = bagData(x_test, y_test, subinstanceLen, subinstanceStride)
    np.save(outDir + '/x_test.npy', x)
    np.save(outDir + '/y_test.npy', y)
    print('Num test %d' % len(x))
    x, y = bagData(x_val, y_val, subinstanceLen, subinstanceStride)
    np.save(outDir + '/x_val.npy', x)
    np.save(outDir + '/y_val.npy', y)
    print('Num val %d' % len(x))
def bagData(X, Y, subinstanceLen, subinstanceStride):
    numClass = 3
    numSteps = 175
    numFeats = 8
    assert X.ndim == 3
    assert X.shape[1] == numSteps
    assert X.shape[2] == numFeats
    assert subinstanceLen <= numSteps
    assert subinstanceLen > 0
    assert subinstanceStride <= numSteps
    assert subinstanceStride >= 0
    assert len(X) == len(Y)
    assert Y.ndim == 2
    assert Y.shape[1] == numClass
    x_bagged = []
    y_bagged = []
    for i, point in enumerate(X[:, :, :]):
        instanceList = []
        start = 0
        end = subinstanceLen
        while True:
            x = point[start:end, :]
            if len(x) < subinstanceLen:
                x_ = np.zeros([subinstanceLen, x.shape[1]])
                x_[:len(x), :] = x[:, :]
                x = x_
            instanceList.append(x)
            if end >= numSteps:
                break
            start += subinstanceStride
            end += subinstanceStride
        bag = np.array(instanceList)
        numSubinstance = bag.shape[0]
        label = Y[i]
        label = np.argmax(label)
        labelBag = np.zeros([numSubinstance, numClass])
        labelBag[:, label] = 1
        x_bagged.append(bag)
        label = np.array(labelBag)
        y_bagged.append(label)
    return np.array(x_bagged), np.array(y_bagged)

In [5]:
subinstanceLen=88
subinstanceStride=30
extractedDir = '/home/deepin/Desktop/projects/hrv/WESAD/'
mkdir('/home/deepin/Desktop/projects/hrv/WESAD/fast_grnn/88_30')
rawDir = extractedDir + '/RAW'
sourceDir = rawDir
outDir = extractedDir + 'fast_grnn' '/%d_%d/' % (subinstanceLen, subinstanceStride)
makeEMIData(subinstanceLen, subinstanceStride, sourceDir, outDir)

Num train 6108
Num test 1697
Num val 679


In [6]:
outDir

'/home/deepin/Desktop/projects/hrv/WESAD/fast_grnn/88_30/'

In [7]:
from __future__ import print_function
import os
import sys
import tensorflow as tf
import numpy as np
os.environ['CUDA_VISIBLE_DEVICES'] ='0'


2023-03-02 09:27:15.366172: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 09:27:16.527333: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-02 09:27:18.246683: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-02 09:27:18.246841: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [8]:
# Network parameters for our FastGRNN + FC Layer
NUM_HIDDEN = 128
NUM_TIMESTEPS = 88
NUM_FEATS = 8
FORGET_BIAS = 1.0
NUM_OUTPUT = 3
USE_DROPOUT = False
KEEP_PROB = 0.9

# Non-linearities can be chosen among "tanh, sigmoid, relu, quantTanh, quantSigm"
UPDATE_NL = "quantTanh"
GATE_NL = "quantSigm"

# Ranks of Parameter matrices for low-rank parameterisation to compress models.
WRANK = 5
URANK = 6

# For dataset API
PREFETCH_NUM = 5
BATCH_SIZE = 175

# Number of epochs in *one iteration*
NUM_EPOCHS = 3

# Number of iterations in *one round*. After each iteration,
# the model is dumped to disk. At the end of the current
# round, the best model among all the dumped models in the
# current round is picked up..
NUM_ITER = 4

# A round consists of multiple training iterations and a belief
# update step using the best model from all of these iterations
NUM_ROUNDS = 6

# A staging direcory to store models
MODEL_PREFIX = '/home/deepin/Desktop/projects/hrv/WESAD/Fast_GRNN/88_30/models/model-fgrnn'

# Loading Data

In [9]:
# Loading the data
path='/home/deepin/Desktop/projects/hrv/WESAD/fast_grnn//88_30/'
x_train, y_train = np.load(path + 'x_train.npy'), np.load(path + 'y_train.npy')
x_test, y_test = np.load(path + 'x_test.npy'), np.load(path + 'y_test.npy')
x_val, y_val = np.load(path + 'x_val.npy'), np.load(path + 'y_val.npy')

# BAG_TEST, BAG_TRAIN, BAG_VAL represent bag_level labels. These are used for the label update
# step of EMI/MI RNN
BAG_TEST = np.argmax(y_test[:, 0, :], axis=1)
BAG_TRAIN = np.argmax(y_train[:, 0, :], axis=1)
BAG_VAL = np.argmax(y_val[:, 0, :], axis=1)
NUM_SUBINSTANCE = x_train.shape[1]
print("x_train shape is:", x_train.shape)
print("y_train shape is:", y_train.shape)
print("x_test shape is:", x_test.shape)
print("y_test shape is:", y_test.shape)

x_train shape is: (6108, 4, 88, 8)
y_train shape is: (6108, 4, 3)
x_test shape is: (1697, 4, 88, 8)
y_test shape is: (1697, 4, 3)


In [10]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT license.

import tensorflow as tf
from tensorflow.python.ops import init_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import variable_scope as vs
from tensorflow.python.ops import gen_math_ops
from tensorflow.python.ops import array_ops
from tensorflow.python.ops.rnn_cell_impl import RNNCell


def gen_non_linearity(A, non_linearity):
    '''
    Returns required activation for a tensor based on the inputs

    non_linearity is either a callable or a value in
        ['tanh', 'sigmoid', 'relu', 'quantTanh', 'quantSigm', 'quantSigm4']
    '''
    if non_linearity == "tanh":
        return math_ops.tanh(A)
    elif non_linearity == "sigmoid":
        return math_ops.sigmoid(A)
    elif non_linearity == "relu":
        return gen_math_ops.maximum(A, 0.0)
    elif non_linearity == "quantTanh":
        return gen_math_ops.maximum(gen_math_ops.minimum(A, 1.0), -1.0)
    elif non_linearity == "quantSigm":
        A = (A + 1.0) / 2.0
        return gen_math_ops.maximum(gen_math_ops.minimum(A, 1.0), 0.0)
    elif non_linearity == "quantSigm4":
        A = (A + 2.0) / 4.0
        return gen_math_ops.maximum(gen_math_ops.minimum(A, 1.0), 0.0)
    else:
        # non_linearity is a user specified function
        if not callable(non_linearity):
            raise ValueError("non_linearity is either a callable or a value " +
                             + "['tanh', 'sigmoid', 'relu', 'quantTanh', " +
                             "'quantSigm'")
        return non_linearity(A)


class FastGRNNCell(RNNCell):
    '''
    FastGRNN Cell with Both Full Rank and Low Rank Formulations
    Has multiple activation functions for the gates
    hidden_size = # hidden units

    gate_non_linearity = nonlinearity for the gate can be chosen from
    [tanh, sigmoid, relu, quantTanh, quantSigm]
    update_non_linearity = nonlinearity for final rnn update
    can be chosen from [tanh, sigmoid, relu, quantTanh, quantSigm]

    wRank = rank of W matrix (creates two matrices if not None)
    uRank = rank of U matrix (creates two matrices if not None)
    zetaInit = init for zeta, the scale param
    nuInit = init for nu, the translation param

    FastGRNN architecture and compression techniques are found in
    FastGRNN(LINK) paper

    Basic architecture is like:

    z_t = gate_nl(Wx_t + Uh_{t-1} + B_g)
    h_t^ = update_nl(Wx_t + Uh_{t-1} + B_h)
    h_t = z_t*h_{t-1} + (sigmoid(zeta)(1-z_t) + sigmoid(nu))*h_t^

    W and U can further parameterised into low rank version by
    W = matmul(W_1, W_2) and U = matmul(U_1, U_2)
    '''

    def __init__(self, hidden_size, gate_non_linearity="sigmoid",
                 update_non_linearity="tanh", wRank=None, uRank=None,
                 zetaInit=1.0, nuInit=-4.0, name="FastGRNN", reuse=None):
        super(FastGRNNCell, self).__init__(_reuse=reuse)
        self._hidden_size = hidden_size
        self._gate_non_linearity = gate_non_linearity
        self._update_non_linearity = update_non_linearity
        self._num_weight_matrices = [1, 1]
        self._wRank = wRank
        self._uRank = uRank
        self._zetaInit = zetaInit
        self._nuInit = nuInit
        if wRank is not None:
            self._num_weight_matrices[0] += 1
        if uRank is not None:
            self._num_weight_matrices[1] += 1
        self._name = name
        self._reuse = reuse

    @property
    def state_size(self):
        return self._hidden_size

    @property
    def output_size(self):
        return self._hidden_size

    @property
    def gate_non_linearity(self):
        return self._gate_non_linearity

    @property
    def update_non_linearity(self):
        return self._update_non_linearity

    @property
    def wRank(self):
        return self._wRank

    @property
    def uRank(self):
        return self._uRank

    @property
    def num_weight_matrices(self):
        return self._num_weight_matrices

    @property
    def name(self):
        return self._name

    @property
    def cellType(self):
        return "FastGRNN"

    def call(self, inputs, state):
        with vs.variable_scope(self._name + "/FastGRNNcell"):

            if self._wRank is None:
                W_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W = vs.get_variable(
                    "W", [inputs.get_shape()[-1], self._hidden_size],
                    initializer=W_matrix_init)
                wComp = math_ops.matmul(inputs, self.W)
            else:
                W_matrix_1_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W1 = vs.get_variable(
                    "W1", [inputs.get_shape()[-1], self._wRank],
                    initializer=W_matrix_1_init)
                W_matrix_2_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W2 = vs.get_variable(
                    "W2", [self._wRank, self._hidden_size],
                    initializer=W_matrix_2_init)
                wComp = math_ops.matmul(
                    math_ops.matmul(inputs, self.W1), self.W2)

            if self._uRank is None:
                U_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U = vs.get_variable(
                    "U", [self._hidden_size, self._hidden_size],
                    initializer=U_matrix_init)
                uComp = math_ops.matmul(state, self.U)
            else:
                U_matrix_1_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U1 = vs.get_variable(
                    "U1", [self._hidden_size, self._uRank],
                    initializer=U_matrix_1_init)
                U_matrix_2_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U2 = vs.get_variable(
                    "U2", [self._uRank, self._hidden_size],
                    initializer=U_matrix_2_init)
                uComp = math_ops.matmul(
                    math_ops.matmul(state, self.U1), self.U2)
            # Init zeta to 6.0 and nu to -6.0 if this doesn't give good
            # results. The inits are hyper-params.
            zeta_init = init_ops.constant_initializer(
                self._zetaInit, dtype=tf.float32)
            self.zeta = vs.get_variable("zeta", [1, 1], initializer=zeta_init)

            nu_init = init_ops.constant_initializer(
                self._nuInit, dtype=tf.float32)
            self.nu = vs.get_variable("nu", [1, 1], initializer=nu_init)

            pre_comp = wComp + uComp

            bias_gate_init = init_ops.constant_initializer(
                1.0, dtype=tf.float32)
            self.bias_gate = vs.get_variable(
                "B_g", [1, self._hidden_size], initializer=bias_gate_init)
            z = gen_non_linearity(pre_comp + self.bias_gate,
                                  self._gate_non_linearity)

            bias_update_init = init_ops.constant_initializer(
                1.0, dtype=tf.float32)
            self.bias_update = vs.get_variable(
                "B_h", [1, self._hidden_size], initializer=bias_update_init)
            c = gen_non_linearity(
                pre_comp + self.bias_update, self._update_non_linearity)
            new_h = z * state + (math_ops.sigmoid(self.zeta) * (1.0 - z) +
                                 math_ops.sigmoid(self.nu)) * c
        return new_h, new_h

    def getVars(self):
        Vars = []
        if self._num_weight_matrices[0] == 1:
            Vars.append(self.W)
        else:
            Vars.extend([self.W1, self.W2])

        if self._num_weight_matrices[1] == 1:
            Vars.append(self.U)
        else:
            Vars.extend([self.U1, self.U2])

        Vars.extend([self.bias_gate, self.bias_update])
        Vars.extend([self.zeta, self.nu])

        return Vars


class FastRNNCell(RNNCell):
    '''
    FastRNN Cell with Both Full Rank and Low Rank Formulations
    Has multiple activation functions for the gates
    hidden_size = # hidden units

    update_non_linearity = nonlinearity for final rnn update
    can be chosen from [tanh, sigmoid, relu, quantTanh, quantSigm]

    wRank = rank of W matrix (creates two matrices if not None)
    uRank = rank of U matrix (creates two matrices if not None)
    alphaInit = init for alpha, the update scalar
    betaInit = init for beta, the weight for previous state

    FastRNN architecture and compression techniques are found in
    FastGRNN(LINK) paper

    Basic architecture is like:

    h_t^ = update_nl(Wx_t + Uh_{t-1} + B_h)
    h_t = sigmoid(beta)*h_{t-1} + sigmoid(alpha)*h_t^

    W and U can further parameterised into low rank version by
    W = matmul(W_1, W_2) and U = matmul(U_1, U_2)
    '''

    def __init__(self, hidden_size, update_non_linearity="tanh",
                 wRank=None, uRank=None, alphaInit=-3.0, betaInit=3.0,
                 name="FastRNN", reuse=None):
        super(FastRNNCell, self).__init__(_reuse=reuse)
        self._hidden_size = hidden_size
        self._update_non_linearity = update_non_linearity
        self._num_weight_matrices = [1, 1]
        self._wRank = wRank
        self._uRank = uRank
        self._alphaInit = alphaInit
        self._betaInit = betaInit
        if wRank is not None:
            self._num_weight_matrices[0] += 1
        if uRank is not None:
            self._num_weight_matrices[1] += 1
        self._name = name
        self._reuse = reuse

    @property
    def state_size(self):
        return self._hidden_size

    @property
    def output_size(self):
        return self._hidden_size

    @property
    def update_non_linearity(self):
        return self._update_non_linearity

    @property
    def wRank(self):
        return self._wRank

    @property
    def uRank(self):
        return self._uRank

    @property
    def num_weight_matrices(self):
        return self._num_weight_matrices

    @property
    def name(self):
        return self._name

    @property
    def cellType(self):
        return "FastRNN"

    def call(self, inputs, state):
        with vs.variable_scope(self._name + "/FastRNNcell"):

            if self._wRank is None:
                W_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W = vs.get_variable(
                    "W", [inputs.get_shape()[-1], self._hidden_size],
                    initializer=W_matrix_init)
                wComp = math_ops.matmul(inputs, self.W)
            else:
                W_matrix_1_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W1 = vs.get_variable(
                    "W1", [inputs.get_shape()[-1], self._wRank],
                    initializer=W_matrix_1_init)
                W_matrix_2_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W2 = vs.get_variable(
                    "W2", [self._wRank, self._hidden_size],
                    initializer=W_matrix_2_init)
                wComp = math_ops.matmul(
                    math_ops.matmul(inputs, self.W1), self.W2)

            if self._uRank is None:
                U_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U = vs.get_variable(
                    "U", [self._hidden_size, self._hidden_size],
                    initializer=U_matrix_init)
                uComp = math_ops.matmul(state, self.U)
            else:
                U_matrix_1_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U1 = vs.get_variable(
                    "U1", [self._hidden_size, self._uRank],
                    initializer=U_matrix_1_init)
                U_matrix_2_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U2 = vs.get_variable(
                    "U2", [self._uRank, self._hidden_size],
                    initializer=U_matrix_2_init)
                uComp = math_ops.matmul(
                    math_ops.matmul(state, self.U1), self.U2)

            alpha_init = init_ops.constant_initializer(
                self._alphaInit, dtype=tf.float32)
            self.alpha = vs.get_variable(
                "alpha", [1, 1], initializer=alpha_init)

            beta_init = init_ops.constant_initializer(
                self._betaInit, dtype=tf.float32)
            self.beta = vs.get_variable("beta", [1, 1], initializer=beta_init)

            pre_comp = wComp + uComp

            bias_update_init = init_ops.constant_initializer(
                1.0, dtype=tf.float32)
            self.bias_update = vs.get_variable(
                "B_h", [1, self._hidden_size], initializer=bias_update_init)
            c = gen_non_linearity(
                pre_comp + self.bias_update, self._update_non_linearity)

            new_h = math_ops.sigmoid(self.beta) * \
                state + math_ops.sigmoid(self.alpha) * c
        return new_h, new_h

    def getVars(self):
        Vars = []
        if self._num_weight_matrices[0] == 1:
            Vars.append(self.W)
        else:
            Vars.extend([self.W1, self.W2])

        if self._num_weight_matrices[1] == 1:
            Vars.append(self.U)
        else:
            Vars.extend([self.U1, self.U2])

        Vars.extend([self.bias_update])
        Vars.extend([self.alpha, self.beta])

        return Vars


class LSTMLRCell(RNNCell):
    '''
    LR - Low Rank
    LSTM LR Cell with Both Full Rank and Low Rank Formulations
    Has multiple activation functions for the gates
    hidden_size = # hidden units

    gate_non_linearity = nonlinearity for the gate can be chosen from
    [tanh, sigmoid, relu, quantTanh, quantSigm]
    update_non_linearity = nonlinearity for final rnn update
    can be chosen from [tanh, sigmoid, relu, quantTanh, quantSigm]

    wRank = rank of all W matrices
    (creates 5 matrices if not None else creates 4 matrices)
    uRank = rank of all U matrices
    (creates 5 matrices if not None else creates 4 matrices)

    LSTM architecture and compression techniques are found in
    LSTM paper

    Basic architecture is like:

    f_t = gate_nl(W1x_t + U1h_{t-1} + B_f)
    i_t = gate_nl(W2x_t + U2h_{t-1} + B_i)
    C_t^ = update_nl(W3x_t + U3h_{t-1} + B_c)
    o_t = gate_nl(W4x_t + U4h_{t-1} + B_o)
    C_t = f_t*C_{t-1} + i_t*C_t^
    h_t = o_t*update_nl(C_t)

    Wi and Ui can further parameterised into low rank version by
    Wi = matmul(W, W_i) and Ui = matmul(U, U_i)
    '''

    def __init__(self, hidden_size, gate_non_linearity="sigmoid",
                 update_non_linearity="tanh", wRank=None, uRank=None,
                 name="LSTMLR", reuse=None):
        super(LSTMLRCell, self).__init__(_reuse=reuse)
        self._hidden_size = hidden_size
        self._gate_non_linearity = gate_non_linearity
        self._update_non_linearity = update_non_linearity
        self._num_weight_matrices = [4, 4]
        self._wRank = wRank
        self._uRank = uRank
        if wRank is not None:
            self._num_weight_matrices[0] += 1
        if uRank is not None:
            self._num_weight_matrices[1] += 1
        self._name = name
        self._reuse = reuse

    @property
    def state_size(self):
        return 2 * self._hidden_size

    @property
    def output_size(self):
        return self._hidden_size

    @property
    def gate_non_linearity(self):
        return self._gate_non_linearity

    @property
    def update_non_linearity(self):
        return self._update_non_linearity

    @property
    def wRank(self):
        return self._wRank

    @property
    def uRank(self):
        return self._uRank

    @property
    def num_weight_matrices(self):
        return self._num_weight_matrices

    @property
    def name(self):
        return self._name

    @property
    def cellType(self):
        return "LSTMLR"

    def call(self, inputs, state):
        c, h = array_ops.split(value=state, num_or_size_splits=2, axis=1)
        with vs.variable_scope(self._name + "/LSTMLRCell"):

            if self._wRank is None:
                W1_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W1 = vs.get_variable(
                    "W1", [inputs.get_shape()[-1], self._hidden_size],
                    initializer=W1_matrix_init)
                W2_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W2 = vs.get_variable(
                    "W2", [inputs.get_shape()[-1], self._hidden_size],
                    initializer=W2_matrix_init)
                W3_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W3 = vs.get_variable(
                    "W3", [inputs.get_shape()[-1], self._hidden_size],
                    initializer=W3_matrix_init)
                W4_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W4 = vs.get_variable(
                    "W4", [inputs.get_shape()[-1], self._hidden_size],
                    initializer=W4_matrix_init)
                wComp1 = math_ops.matmul(inputs, self.W1)
                wComp2 = math_ops.matmul(inputs, self.W2)
                wComp3 = math_ops.matmul(inputs, self.W3)
                wComp4 = math_ops.matmul(inputs, self.W4)
            else:
                W_matrix_r_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W = vs.get_variable(
                    "W", [inputs.get_shape()[-1], self._wRank],
                    initializer=W_matrix_r_init)
                W1_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W1 = vs.get_variable(
                    "W1", [self._wRank, self._hidden_size],
                    initializer=W1_matrix_init)
                W2_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W2 = vs.get_variable(
                    "W2", [self._wRank, self._hidden_size],
                    initializer=W2_matrix_init)
                W3_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W3 = vs.get_variable(
                    "W3", [self._wRank, self._hidden_size],
                    initializer=W3_matrix_init)
                W4_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W4 = vs.get_variable(
                    "W4", [self._wRank, self._hidden_size],
                    initializer=W4_matrix_init)
                wComp1 = math_ops.matmul(
                    math_ops.matmul(inputs, self.W), self.W1)
                wComp2 = math_ops.matmul(
                    math_ops.matmul(inputs, self.W), self.W2)
                wComp3 = math_ops.matmul(
                    math_ops.matmul(inputs, self.W), self.W3)
                wComp4 = math_ops.matmul(
                    math_ops.matmul(inputs, self.W), self.W4)
            if self._uRank is None:
                U1_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U1 = vs.get_variable(
                    "U1", [self._hidden_size, self._hidden_size],
                    initializer=U1_matrix_init)
                U2_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U2 = vs.get_variable(
                    "U2", [self._hidden_size, self._hidden_size],
                    initializer=U2_matrix_init)
                U3_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U3 = vs.get_variable(
                    "U3", [self._hidden_size, self._hidden_size],
                    initializer=U3_matrix_init)
                U4_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U4 = vs.get_variable(
                    "U4", [self._hidden_size, self._hidden_size],
                    initializer=U4_matrix_init)
                uComp1 = math_ops.matmul(h, self.U1)
                uComp2 = math_ops.matmul(h, self.U2)
                uComp3 = math_ops.matmul(h, self.U3)
                uComp4 = math_ops.matmul(h, self.U4)
            else:
                U_matrix_r_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U = vs.get_variable(
                    "U", [self._hidden_size, self._uRank],
                    initializer=U_matrix_r_init)
                U1_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U1 = vs.get_variable(
                    "U1", [self._uRank, self._hidden_size],
                    initializer=U1_matrix_init)
                U2_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U2 = vs.get_variable(
                    "U2", [self._uRank, self._hidden_size],
                    initializer=U2_matrix_init)
                U3_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U3 = vs.get_variable(
                    "U3", [self._uRank, self._hidden_size],
                    initializer=U3_matrix_init)
                U4_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U4 = vs.get_variable(
                    "U4", [self._uRank, self._hidden_size],
                    initializer=U4_matrix_init)

                uComp1 = math_ops.matmul(
                    math_ops.matmul(h, self.U), self.U1)
                uComp2 = math_ops.matmul(
                    math_ops.matmul(h, self.U), self.U2)
                uComp3 = math_ops.matmul(
                    math_ops.matmul(h, self.U), self.U3)
                uComp4 = math_ops.matmul(
                    math_ops.matmul(h, self.U), self.U4)

            pre_comp1 = wComp1 + uComp1
            pre_comp2 = wComp2 + uComp2
            pre_comp3 = wComp3 + uComp3
            pre_comp4 = wComp4 + uComp4

            bias_gate_init = init_ops.constant_initializer(
                1.0, dtype=tf.float32)
            self.bias_f = vs.get_variable(
                "B_f", [1, self._hidden_size], initializer=bias_gate_init)
            self.bias_i = vs.get_variable(
                "B_i", [1, self._hidden_size], initializer=bias_gate_init)
            self.bias_c = vs.get_variable(
                "B_c", [1, self._hidden_size], initializer=bias_gate_init)
            self.bias_o = vs.get_variable(
                "B_o", [1, self._hidden_size], initializer=bias_gate_init)

            f = gen_non_linearity(pre_comp1 + self.bias_f,
                                  self._gate_non_linearity)
            i = gen_non_linearity(pre_comp2 + self.bias_i,
                                  self._gate_non_linearity)
            o = gen_non_linearity(pre_comp4 + self.bias_o,
                                  self._gate_non_linearity)

            c_ = gen_non_linearity(
                pre_comp3 + self.bias_c, self._update_non_linearity)

            new_c = f * c + i * c_
            new_h = o * gen_non_linearity(new_c, self._update_non_linearity)
            new_state = array_ops.concat([new_c, new_h], 1)

        return new_h, new_state

    def getVars(self):
        Vars = []
        if self._num_weight_matrices[0] == 4:
            Vars.extend([self.W1, self.W2, self.W3, self.W4])
        else:
            Vars.extend([self.W, self.W1, self.W2, self.W3, self.W4])

        if self._num_weight_matrices[1] == 4:
            Vars.extend([self.U1, self.U2, self.U3, self.U4])
        else:
            Vars.extend([self.U, self.U1, self.U2, self.U3, self.U4])

        Vars.extend([self.bias_f, self.bias_i, self.bias_c, self.bias_o])

        return Vars


class GRULRCell(RNNCell):
    '''
    GRU LR Cell with Both Full Rank and Low Rank Formulations
    Has multiple activation functions for the gates
    hidden_size = # hidden units

    gate_non_linearity = nonlinearity for the gate can be chosen from
    [tanh, sigmoid, relu, quantTanh, quantSigm]
    update_non_linearity = nonlinearity for final rnn update
    can be chosen from [tanh, sigmoid, relu, quantTanh, quantSigm]

    wRank = rank of W matrix
    (creates 4 matrices if not None else creates 3 matrices)
    uRank = rank of U matrix
    (creates 4 matrices if not None else creates 3 matrices)

    GRU architecture and compression techniques are found in
    GRU(LINK) paper

    Basic architecture is like:

    r_t = gate_nl(W1x_t + U1h_{t-1} + B_r)
    z_t = gate_nl(W2x_t + U2h_{t-1} + B_g)
    h_t^ = update_nl(W3x_t + r_t*U3(h_{t-1}) + B_h)
    h_t = z_t*h_{t-1} + (1-z_t)*h_t^

    Wi and Ui can further parameterised into low rank version by
    Wi = matmul(W, W_i) and Ui = matmul(U, U_i)
    '''

    def __init__(self, hidden_size, gate_non_linearity="sigmoid",
                 update_non_linearity="tanh", wRank=None, uRank=None,
                 name="GRULR", reuse=None):
        super(GRULRCell, self).__init__(_reuse=reuse)
        self._hidden_size = hidden_size
        self._gate_non_linearity = gate_non_linearity
        self._update_non_linearity = update_non_linearity
        self._num_weight_matrices = [3, 3]
        self._wRank = wRank
        self._uRank = uRank
        if wRank is not None:
            self._num_weight_matrices[0] += 1
        if uRank is not None:
            self._num_weight_matrices[1] += 1
        self._name = name
        self._reuse = reuse

    @property
    def state_size(self):
        return self._hidden_size

    @property
    def output_size(self):
        return self._hidden_size

    @property
    def gate_non_linearity(self):
        return self._gate_non_linearity

    @property
    def update_non_linearity(self):
        return self._update_non_linearity

    @property
    def wRank(self):
        return self._wRank

    @property
    def uRank(self):
        return self._uRank

    @property
    def num_weight_matrices(self):
        return self._num_weight_matrices

    @property
    def name(self):
        return self._name

    @property
    def cellType(self):
        return "GRULR"

    def call(self, inputs, state):
        with vs.variable_scope(self._name + "/GRULRCell"):

            if self._wRank is None:
                W1_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W1 = vs.get_variable(
                    "W1", [inputs.get_shape()[-1], self._hidden_size],
                    initializer=W1_matrix_init)
                W2_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W2 = vs.get_variable(
                    "W2", [inputs.get_shape()[-1], self._hidden_size],
                    initializer=W2_matrix_init)
                W3_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W3 = vs.get_variable(
                    "W3", [inputs.get_shape()[-1], self._hidden_size],
                    initializer=W3_matrix_init)
                wComp1 = math_ops.matmul(inputs, self.W1)
                wComp2 = math_ops.matmul(inputs, self.W2)
                wComp3 = math_ops.matmul(inputs, self.W3)
            else:
                W_matrix_r_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W = vs.get_variable(
                    "W", [inputs.get_shape()[-1], self._wRank],
                    initializer=W_matrix_r_init)
                W1_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W1 = vs.get_variable(
                    "W1", [self._wRank, self._hidden_size],
                    initializer=W1_matrix_init)
                W2_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W2 = vs.get_variable(
                    "W2", [self._wRank, self._hidden_size],
                    initializer=W2_matrix_init)
                W3_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W3 = vs.get_variable(
                    "W3", [self._wRank, self._hidden_size],
                    initializer=W3_matrix_init)
                wComp1 = math_ops.matmul(
                    math_ops.matmul(inputs, self.W), self.W1)
                wComp2 = math_ops.matmul(
                    math_ops.matmul(inputs, self.W), self.W2)
                wComp3 = math_ops.matmul(
                    math_ops.matmul(inputs, self.W), self.W3)

            if self._uRank is None:
                U1_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U1 = vs.get_variable(
                    "U1", [self._hidden_size, self._hidden_size],
                    initializer=U1_matrix_init)
                U2_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U2 = vs.get_variable(
                    "U2", [self._hidden_size, self._hidden_size],
                    initializer=U2_matrix_init)
                U3_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U3 = vs.get_variable(
                    "U3", [self._hidden_size, self._hidden_size],
                    initializer=U3_matrix_init)
                uComp1 = math_ops.matmul(state, self.U1)
                uComp2 = math_ops.matmul(state, self.U2)
            else:
                U_matrix_r_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U = vs.get_variable(
                    "U", [self._hidden_size, self._uRank],
                    initializer=U_matrix_r_init)
                U1_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U1 = vs.get_variable(
                    "U1", [self._uRank, self._hidden_size],
                    initializer=U1_matrix_init)
                U2_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U2 = vs.get_variable(
                    "U2", [self._uRank, self._hidden_size],
                    initializer=U2_matrix_init)
                U3_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U3 = vs.get_variable(
                    "U3", [self._uRank, self._hidden_size],
                    initializer=U3_matrix_init)
                uComp1 = math_ops.matmul(
                    math_ops.matmul(state, self.U), self.U1)
                uComp2 = math_ops.matmul(
                    math_ops.matmul(state, self.U), self.U2)

            pre_comp1 = wComp1 + uComp1
            pre_comp2 = wComp2 + uComp2

            bias_r_init = init_ops.constant_initializer(
                1.0, dtype=tf.float32)
            self.bias_r = vs.get_variable(
                "B_r", [1, self._hidden_size], initializer=bias_r_init)
            r = gen_non_linearity(pre_comp1 + self.bias_r,
                                  self._gate_non_linearity)

            bias_gate_init = init_ops.constant_initializer(
                1.0, dtype=tf.float32)
            self.bias_gate = vs.get_variable(
                "B_g", [1, self._hidden_size], initializer=bias_gate_init)
            z = gen_non_linearity(pre_comp2 + self.bias_gate,
                                  self._gate_non_linearity)

            if self._uRank is None:
                pre_comp3 = wComp3 + math_ops.matmul(r * state, self.U3)
            else:
                pre_comp3 = wComp3 + \
                    math_ops.matmul(math_ops.matmul(
                        r * state, self.U), self.U3)

            bias_update_init = init_ops.constant_initializer(
                1.0, dtype=tf.float32)
            self.bias_update = vs.get_variable(
                "B_h", [1, self._hidden_size], initializer=bias_update_init)
            c = gen_non_linearity(
                pre_comp3 + self.bias_update, self._update_non_linearity)

            new_h = z * state + (1.0 - z) * c

        return new_h, new_h

    def getVars(self):
        Vars = []
        if self._num_weight_matrices[0] == 3:
            Vars.extend([self.W1, self.W2, self.W3])
        else:
            Vars.extend([self.W, self.W1, self.W2, self.W3])

        if self._num_weight_matrices[1] == 3:
            Vars.extend([self.U1, self.U2, self.U3])
        else:
            Vars.extend([self.U, self.U1, self.U2, self.U3])

        Vars.extend([self.bias_r, self.bias_gate, self.bias_update])

        return Vars


class UGRNNLRCell(RNNCell):
    '''
    UGRNN LR Cell with Both Full Rank and Low Rank Formulations
    Has multiple activation functions for the gates
    hidden_size = # hidden units

    gate_non_linearity = nonlinearity for the gate can be chosen from
    [tanh, sigmoid, relu, quantTanh, quantSigm]
    update_non_linearity = nonlinearity for final rnn update
    can be chosen from [tanh, sigmoid, relu, quantTanh, quantSigm]

    wRank = rank of W matrix
    (creates 3 matrices if not None else creates 2 matrices)
    uRank = rank of U matrix
    (creates 3 matrices if not None else creates 2 matrices)

    UGRNN architecture and compression techniques are found in
    UGRNN(LINK) paper

    Basic architecture is like:

    z_t = gate_nl(W1x_t + U1h_{t-1} + B_g)
    h_t^ = update_nl(W1x_t + U1h_{t-1} + B_h)
    h_t = z_t*h_{t-1} + (1-z_t)*h_t^

    Wi and Ui can further parameterised into low rank version by
    Wi = matmul(W, W_i) and Ui = matmul(U, U_i)
    '''

    def __init__(self, hidden_size, gate_non_linearity="sigmoid",
                 update_non_linearity="tanh", wRank=None, uRank=None,
                 name="UGRNNLR", reuse=None):
        super(UGRNNLRCell, self).__init__(_reuse=reuse)
        self._hidden_size = hidden_size
        self._gate_non_linearity = gate_non_linearity
        self._update_non_linearity = update_non_linearity
        self._num_weight_matrices = [2, 2]
        self._wRank = wRank
        self._uRank = uRank
        if wRank is not None:
            self._num_weight_matrices[0] += 1
        if uRank is not None:
            self._num_weight_matrices[1] += 1
        self._name = name
        self._reuse = reuse

    @property
    def state_size(self):
        return self._hidden_size

    @property
    def output_size(self):
        return self._hidden_size

    @property
    def gate_non_linearity(self):
        return self._gate_non_linearity

    @property
    def update_non_linearity(self):
        return self._update_non_linearity

    @property
    def wRank(self):
        return self._wRank

    @property
    def uRank(self):
        return self._uRank

    @property
    def num_weight_matrices(self):
        return self._num_weight_matrices

    @property
    def name(self):
        return self._name

    @property
    def cellType(self):
        return "UGRNNLR"

    def call(self, inputs, state):
        with vs.variable_scope(self._name + "/UGRNNLRCell"):

            if self._wRank is None:
                W1_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W1 = vs.get_variable(
                    "W1", [inputs.get_shape()[-1], self._hidden_size],
                    initializer=W1_matrix_init)
                W2_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W2 = vs.get_variable(
                    "W2", [inputs.get_shape()[-1], self._hidden_size],
                    initializer=W2_matrix_init)
                wComp1 = math_ops.matmul(inputs, self.W1)
                wComp2 = math_ops.matmul(inputs, self.W2)
            else:
                W_matrix_r_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W = vs.get_variable(
                    "W", [inputs.get_shape()[-1], self._wRank],
                    initializer=W_matrix_r_init)
                W1_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W1 = vs.get_variable(
                    "W1", [self._wRank, self._hidden_size],
                    initializer=W1_matrix_init)
                W2_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.W2 = vs.get_variable(
                    "W2", [self._wRank, self._hidden_size],
                    initializer=W2_matrix_init)
                wComp1 = math_ops.matmul(
                    math_ops.matmul(inputs, self.W), self.W1)
                wComp2 = math_ops.matmul(
                    math_ops.matmul(inputs, self.W), self.W2)

            if self._uRank is None:
                U1_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U1 = vs.get_variable(
                    "U1", [self._hidden_size, self._hidden_size],
                    initializer=U1_matrix_init)
                U2_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U2 = vs.get_variable(
                    "U2", [self._hidden_size, self._hidden_size],
                    initializer=U2_matrix_init)
                uComp1 = math_ops.matmul(state, self.U1)
                uComp2 = math_ops.matmul(state, self.U2)
            else:
                U_matrix_r_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U = vs.get_variable(
                    "U", [self._hidden_size, self._uRank],
                    initializer=U_matrix_r_init)
                U1_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U1 = vs.get_variable(
                    "U1", [self._uRank, self._hidden_size],
                    initializer=U1_matrix_init)
                U2_matrix_init = init_ops.random_normal_initializer(
                    mean=0.0, stddev=0.1, dtype=tf.float32)
                self.U2 = vs.get_variable(
                    "U2", [self._uRank, self._hidden_size],
                    initializer=U2_matrix_init)
                uComp1 = math_ops.matmul(
                    math_ops.matmul(state, self.U), self.U1)
                uComp2 = math_ops.matmul(
                    math_ops.matmul(state, self.U), self.U2)

            pre_comp1 = wComp1 + uComp1
            pre_comp2 = wComp2 + uComp2

            bias_gate_init = init_ops.constant_initializer(
                1.0, dtype=tf.float32)
            self.bias_gate = vs.get_variable(
                "B_g", [1, self._hidden_size], initializer=bias_gate_init)
            z = gen_non_linearity(pre_comp1 + self.bias_gate,
                                  self._gate_non_linearity)

            bias_update_init = init_ops.constant_initializer(
                1.0, dtype=tf.float32)
            self.bias_update = vs.get_variable(
                "B_h", [1, self._hidden_size], initializer=bias_update_init)
            c = gen_non_linearity(
                pre_comp2 + self.bias_update, self._update_non_linearity)

            new_h = z * state + (1.0 - z) * c

        return new_h, new_h

    def getVars(self):
        Vars = []
        if self._num_weight_matrices[0] == 2:
            Vars.extend([self.W1, self.W2])
        else:
            Vars.extend([self.W, self.W1, self.W2])

        if self._num_weight_matrices[1] == 2:
            Vars.extend([self.U1, self.U2])
        else:
            Vars.extend([self.U, self.U1, self.U2])

        Vars.extend([self.bias_gate, self.bias_update])

        return Vars


class EMI_DataPipeline():
    '''
    The data input block for EMI-RNN training. Since EMI-RNN is an expensive
    algorithm due to the multiple rounds of updates that are to be performed,
    we avoid using feed dict to feed data into tensorflow and rather,
    exploit the dataset API. This class abstracts away most of the dataset API
    implementation details and provides a module that ingests data in numpy
    matrices and serves them to the remainder of the computation graph.

    This class uses reinitializable iterators. Please refer to the dataset API
    docs for more information.

    This class supports resuming from checkpoint files. Provide the restored
    meta graph as an argument to __init__ to enable this behaviour.

    Usage:
        Step 1: Create a data input pipeline object and obtain the x_batch and
        y_batch tensors. These should be fed to other parts of the graph that
        are supposed to act on the input data.
        ```
            inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS,
                                             NUM_FEATS, NUM_OUTPUT)
            x_batch, y_batch = inputPipeline()
            # feed to emiLSTM or some other computation subgraph
            y_cap = emiLSTM(x_batch)
        ```

        Step 2:  Create other parts of the computation graph (loss operations,
        training ops etc). After the graph construction is complete and after
        initializing the Tensorflow graph with global_variables_initializer,
        initialize the iterator with the input data by calling:
            inputPipeline.runInitializer(x_train, y_trian, ...)

        Step 3: You can now iterate over batches by running some computation
        operation as you would normally do in seesion.run(..). Att the end of
        the data, tf.errors.OutOfRangeError will be
        thrown.
        ```
        while True:
            try:
                sess.run(y_cap)
            except tf.errors.OutOfRangeError:
                break
        ```
    '''

    def __init__(self, numSubinstance, numTimesteps, numFeats, numOutput,
                 graph=None, prefetchNum=5):
        '''
        numSubinstance, numTimeSteps, numFeats, numOutput:
            Dataset characteristics. Please refer to the data preparation
            documentation for more information provided in `examples/EMI-RNN`
        graph: This module supports resuming/restoring from a saved metagraph. To
            enable this behaviour, pass the restored graph as an argument. A
            saved metagraph can be restored using the edgeml.utils.GraphManager
            module.
        prefetchNum: The number of asynchronous prefetch to do when iterating over
            the data. Please refer to 'prefetching' in Tensorflow dataset API
        '''

        self.numSubinstance = numSubinstance
        self.numTimesteps = numTimesteps
        self.numFeats = numFeats
        self.graph = graph
        self.prefetchNum = prefetchNum
        self.numOutput = numOutput
        self.graphCreated = False
        # Either restore or create the following
        self.X = None
        self.Y = None
        self.batchSize = None
        self.numEpochs = None
        self.dataset_init = None
        self.x_batch = None
        self.y_batch = None
        # Internal
        self.scope = 'EMI/'

    def _createGraph(self):
        assert self.graphCreated is False
        dim = [None, self.numSubinstance, self.numTimesteps, self.numFeats]
        scope = self.scope + 'input-pipeline/'
        with tf.name_scope(scope):
            X = tf.compat.v1.placeholder(tf.float32, dim, name='inpX')
            Y = tf.compat.v1.placeholder(tf.float32, [None, self.numSubinstance,
                                        self.numOutput], name='inpY')
            batchSize = tf.compat.v1.placeholder(tf.int64, name='batch-size')
            numEpochs = tf.compat.v1.placeholder(tf.int64, name='num-epochs')

            dataset_x_target = tf.data.Dataset.from_tensor_slices(X)
            dataset_y_target = tf.data.Dataset.from_tensor_slices(Y)
            couple = (dataset_x_target, dataset_y_target)
            ds_target = tf.data.Dataset.zip(couple).repeat(numEpochs)
            ds_target = ds_target.batch(batchSize)
            ds_target = ds_target.prefetch(self.prefetchNum)
            ds_iterator_target = tf.compat.v1.data.make_initializable_iterator(ds_target)
            x_batch, y_batch = ds_iterator_target.get_next()
            tf.compat.v1.add_to_collection('next-x-batch', x_batch)
            tf.compat.v1.add_to_collection('next-y-batch', y_batch)
        self.X = X
        self.Y = Y
        self.batchSize = batchSize
        self.numEpochs = numEpochs
        self.dataset_init = ds_iterator_target.initializer
        self.x_batch, self.y_batch = x_batch, y_batch
        self.graphCreated = True

    def _restoreGraph(self, graph):
        assert self.graphCreated is False
        scope = 'EMI/input-pipeline/'
        self.X = graph.get_tensor_by_name(scope + "inpX:0")
        self.Y = graph.get_tensor_by_name(scope + "inpY:0")
        self.batchSize = graph.get_tensor_by_name(scope + "batch-size:0")
        self.numEpochs = graph.get_tensor_by_name(scope + "num-epochs:0")
        self.dataset_init = graph.get_operation_by_name(scope + "dataset-init")
        self.x_batch = graph.get_collection('next-x-batch')
        self.y_batch = graph.get_collection('next-y-batch')
        msg = 'More than one tensor named next-x-batch/next-y-batch. '
        msg += 'Are you not resetting your graph?'
        assert len(self.x_batch) == 1, msg
        assert len(self.y_batch) == 1, msg
        self.x_batch = self.x_batch[0]
        self.y_batch = self.y_batch[0]
        self.graphCreated = True

    def __call__(self):
        '''
        The call method performs graph construction either by
        creating a new graph or, if a restored meta graph is provided, by
        restoring operators from this meta graph.

        returns iterators (x_batch, y_batch)
        '''
        if self.graphCreated is True:
            return self.x_batch, self.y_batch
        if self.graph is None:
            self._createGraph()
        else:
            self._restoreGraph(self.graph)
        assert self.graphCreated is True
        return self.x_batch, self.y_batch

    def restoreFromGraph(self, graph, *args, **kwargs):
        '''
        This method provides an alternate way of restoring
        from a saved meta graph - without having to provide the restored meta
        graph as a parameter to __init__. This is useful when, in between
        training, you want to reset the entire computation graph and reload a
        new meta graph from disk. This method allows you to attach to this
        newly loaded meta graph without having to create a new EMI_DataPipeline
        object. Use this method only when you want to clear/reset the existing
        computational graph.
        '''
        self.graphCreated = False
        self.graph = graph
        self._restoreGraph(graph)
        assert self.graphCreated is True

    def runInitializer(self, sess, x_data, y_data, batchSize, numEpochs):
        '''
        This method is used to ingest data by the dataset API. Call this method
        with the data matrices after the graph has been initialized.

        x_data, y_data, batchSize: Self explanatory.
        numEpochs: The Tensorflow dataset API implements iteration over epochs
            by appending the data to itself numEpochs times and then iterating
            over the resulting data as if it was a single data set.
        '''
        assert self.graphCreated is True
        msg = 'X shape should be [-1, numSubinstance, numTimesteps, numFeats]'
        assert x_data.ndim == 4, msg
        assert x_data.shape[1] == self.numSubinstance, msg
        assert x_data.shape[2] == self.numTimesteps, msg
        assert x_data.shape[3] == self.numFeats, msg
        msg = 'X and Y sould have same first dimension'
        assert y_data.shape[0] == x_data.shape[0], msg
        msg = 'Y shape should be [-1, numSubinstance, numOutput]'
        assert y_data.shape[1] == self.numSubinstance, msg
        assert y_data.shape[2] == self.numOutput, msg
        feed_dict = {
            self.X: x_data,
            self.Y: y_data,
            self.batchSize: batchSize,
            self.numEpochs: numEpochs
        }
        assert self.dataset_init is not None, 'Internal error!'
        sess.run(self.dataset_init, feed_dict=feed_dict)


class EMI_RNN():

    def __init__(self, *args, **kwargs):
        """
        Abstract base class for RNN architectures compatible with EMI-RNN.
        This class is extended by specific architectures like LSTM/GRU/FastGRNN
        etc.

        Note: We are not using the PEP recommended abc module since it is
        difficult to support in both python 2 and 3 """
        self.graphCreated = False
        # Model specific matrices, parameter should be saved
        self.graph = None
        self.varList = []
        self.output = None
        self.assignOps = []
        raise NotImplementedError("This is intended to act similar to an " +
                                  "abstract class. Instantiating is not " +
                                  "allowed.")

    def __call__(self, x_batch, **kwargs):
        '''
        The call method performs graph construction either by
        creating a new graph or, if a restored meta graph is provided, by
        restoring operators from this meta graph.

        x_batch: Dataset API iterators to the data.

        returns forward computation output tensor
        '''
        if self.graphCreated is True:
            assert self.output is not None
            return self.output
        if self.graph is None:
            output = self._createBaseGraph(x_batch, **kwargs)
            assert self.graphCreated is False
            self._createExtendedGraph(output, **kwargs)
        else:
            self._restoreBaseGraph(self.graph, **kwargs)
            assert self.graphCreated is False
            self._restoreExtendedGraph(self.graph, **kwargs)
        assert self.graphCreated is True
        return self.output

    def restoreFromGraph(self, graph, **kwargs):
        '''
        This method provides an alternate way of restoring
        from a saved meta graph - without having to provide the restored meta
        graph as a parameter to __init__. This is useful when, in between
        training, you want to reset the entire computation graph and reload a
        new meta graph from disk. This method allows you to attach to this
        newly loaded meta graph without having to create a new EMI_DataPipeline
        object. Use this method only when you want to clear/reset the existing
        computational graph.
        '''
        self.graphCreated = False
        self.varList = []
        self.output = None
        self.assignOps = []
        self.graph = graph
        self._restoreBaseGraph(self.graph, **kwargs)
        assert self.graphCreated is False
        self._restoreExtendedGraph(self.graph, **kwargs)
        assert self.graphCreated is True

    def getModelParams(self):
        raise NotImplementedError("Subclass does not implement this method")

    def _createBaseGraph(self, x_batch, **kwargs):
        raise NotImplementedError("Subclass does not implement this method")

    def _createExtendedGraph(self, baseOutput, **kwargs):
        raise NotImplementedError("Subclass does not implement this method")

    def _restoreBaseGraph(self, graph, **kwargs):
        raise NotImplementedError("Subclass does not implement this method")

    def _restoreExtendedGraph(self, graph, **kwargs):
        raise NotImplementedError("Subclass does not implement this method")

    def addBaseAssignOps(self, graph, initVarList, **kwargs):
        raise NotImplementedError("Subclass does not implement this method")

    def addExtendedAssignOps(self, graph, **kwargs):
        raise NotImplementedError("Subclass does not implement this method")


class EMI_BasicLSTM(EMI_RNN):

    def __init__(self, numSubinstance, numHidden, numTimeSteps,
                 numFeats, graph=None, forgetBias=1.0, useDropout=False):
        '''
        EMI-RNN using LSTM cell. The architecture consists of a single LSTM
        layer followed by a secondary classifier. The secondary classifier is
        not defined as part of this module and is left for the user to define,
        through the redefinition of the '_createExtendedGraph' and
        '_restoreExtendedGraph' methods.

        This class supports restoring from a meta-graph. Provide the restored
        graph as an argument to the graph keyword to enable this behaviour.

        numSubinstance: Number of sub-instance.
        numHidden: The dimension of the hidden state.
        numTimeSteps: The number of time steps of the RNN.
        numFeats: The feature vector dimension for each time step.
        graph: A restored metagraph. Provide a graph if restoring form a meta
            graph is required.
        forgetBias: Bias for the forget gate of the LSTM.
        useDropout: Set to True if a dropout layer is to be added between
            inputs and outputs to the LSTM.
        '''
        self.numHidden = numHidden
        self.numTimeSteps = numTimeSteps
        self.numFeats = numFeats
        self.useDropout = useDropout
        self.forgetBias = forgetBias
        self.numSubinstance = numSubinstance
        self.graph = graph
        self.graphCreated = False
        # Restore or initialize
        self.keep_prob = None
        self.varList = []
        self.output = None
        self.assignOps = []
        # Internal
        self._scope = 'EMI/BasicLSTM/'

    def _createBaseGraph(self, X, **kwargs):
        assert self.graphCreated is False
        msg = 'X should be of form [-1, numSubinstance, numTimeSteps, numFeatures]'
        assert X.get_shape().ndims == 4, msg
        assert X.shape[1] == self.numSubinstance
        assert X.shape[2] == self.numTimeSteps
        assert X.shape[3] == self.numFeats
        # Reshape into 3D such that the first dimension is -1 * numSubinstance
        # where each numSubinstance segment corresponds to one bag
        # then shape it back in into 4D
        scope = self._scope
        keep_prob = None
        with tf.name_scope(scope):
            x = tf.reshape(X, [-1, self.numTimeSteps, self.numFeats])
            x = tf.unstack(x, num=self.numTimeSteps, axis=1)
            # Get the LSTM output
            cell = tf.nn.rnn_cell.BasicLSTMCell(self.numHidden,
                                                forget_bias=self.forgetBias,
                                                name='EMI-LSTM-Cell')
            wrapped_cell = cell
            if self.useDropout is True:
                keep_prob = tf.placeholder(dtype=tf.float32, name='keep-prob')
                wrapped_cell = tf.contrib.rnn.DropoutWrapper(cell,
                                                             input_keep_prob=keep_prob,
                                                             output_keep_prob=keep_prob)
            outputs__, states = tf.nn.static_rnn(
                wrapped_cell, x, dtype=tf.float32)
            outputs = []
            for output in outputs__:
                outputs.append(tf.expand_dims(output, axis=1))
            # Convert back to bag form
            outputs = tf.concat(outputs, axis=1, name='concat-output')
            dims = [-1, self.numSubinstance, self.numTimeSteps, self.numHidden]
            output = tf.reshape(outputs, dims, name='bag-output')

        LSTMVars = cell.variables
        self.varList.extend(LSTMVars)
        if self.useDropout:
            self.keep_prob = keep_prob
        self.output = output
        return self.output

    def _restoreBaseGraph(self, graph, **kwargs):
        assert self.graphCreated is False
        assert self.graph is not None
        scope = self._scope
        if self.useDropout:
            self.keep_prob = graph.get_tensor_by_name(scope + 'keep-prob:0')
        self.output = graph.get_tensor_by_name(scope + 'bag-output:0')
        kernel = graph.get_tensor_by_name("rnn/EMI-LSTM-Cell/kernel:0")
        bias = graph.get_tensor_by_name("rnn/EMI-LSTM-Cell/bias:0")
        assert len(self.varList) == 0
        self.varList = [kernel, bias]

    def getModelParams(self):
        '''
        Returns the LSTM kernel and bias tensors.
        returns [kernel, bias]
        '''
        assert self.graphCreated is True, "Graph is not created"
        assert len(self.varList) == 2
        return self.varList

    def addBaseAssignOps(self, graph, initVarList, **kwargs):
        '''
        Adds Tensorflow assignment operations to all of the model tensors.
        These operations can then be used to initialize these tensors from
        numpy matrices by running these operators

        initVarList: A list of numpy matrices that will be used for
            initialization by the assignment operation. For EMI_BasicLSTM, this
            should be [kernel, bias] matrices.
        '''
        assert initVarList is not None
        assert len(initVarList) == 2
        k_ = graph.get_tensor_by_name('rnn/EMI-LSTM-Cell/kernel:0')
        b_ = graph.get_tensor_by_name('rnn/EMI-LSTM-Cell/bias:0')
        kernel, bias = initVarList[-2], initVarList[-1]
        k_op = tf.assign(k_, kernel)
        b_op = tf.assign(b_, bias)
        self.assignOps.extend([k_op, b_op])


class EMI_GRU(EMI_RNN):

    def __init__(self, numSubinstance, numHidden, numTimeSteps,
                 numFeats, graph=None, useDropout=False):
        '''
        EMI-RNN using GRU cell. The architecture consists of a single GRU
        layer followed by a secondary classifier. The secondary classifier is
        not defined as part of this module and is left for the user to define,
        through the redefinition of the '_createExtendedGraph' and
        '_restoreExtendedGraph' methods.

        This class supports restoring from a meta-graph. Provide the restored
        graph as value to the graph keyword to enable this behaviour.

        numSubinstance: Number of sub-instance.
        numHidden: The dimension of the hidden state.
        numTimeSteps: The number of time steps of the RNN.
        numFeats: The feature vector dimension for each time step.
        graph: A restored metagraph. Provide a graph if restoring form a meta
            graph is required.
        useDropout: Set to True if a dropout layer is to be added between
            inputs and outputs to the RNN.
        '''
        self.numHidden = numHidden
        self.numTimeSteps = numTimeSteps
        self.numFeats = numFeats
        self.useDropout = useDropout
        self.numSubinstance = numSubinstance
        self.graph = graph
        self.graphCreated = False
        # Restore or initialize
        self.keep_prob = None
        self.varList = []
        self.output = None
        self.assignOps = []
        # Internal
        self._scope = 'EMI/GRU/'

    def _createBaseGraph(self, X, **kwargs):
        assert self.graphCreated is False
        msg = 'X should be of form [-1, numSubinstance, numTimeSteps, numFeatures]'
        assert X.get_shape().ndims == 4, msg
        assert X.shape[1] == self.numSubinstance
        assert X.shape[2] == self.numTimeSteps
        assert X.shape[3] == self.numFeats
        # Reshape into 3D suself.h that the first dimension is -1 * numSubinstance
        # where each numSubinstance segment corresponds to one bag
        # then shape it back in into 4D
        scope = self._scope
        keep_prob = None
        with tf.name_scope(scope):
            x = tf.reshape(X, [-1, self.numTimeSteps, self.numFeats])
            x = tf.unstack(x, num=self.numTimeSteps, axis=1)
            # Get the GRU output
            cell = tf.nn.rnn_cell.GRUCell(self.numHidden, name='EMI-GRU-Cell')
            wrapped_cell = cell
            if self.useDropout is True:
                keep_prob = tf.placeholder(dtype=tf.float32, name='keep-prob')
                wrapped_cell = tf.contrib.rnn.DropoutWrapper(cell,
                                                             input_keep_prob=keep_prob,
                                                             output_keep_prob=keep_prob)
            outputs__, states = tf.nn.static_rnn(
                wrapped_cell, x, dtype=tf.float32)
            outputs = []
            for output in outputs__:
                outputs.append(tf.expand_dims(output, axis=1))
            # Convert back to bag form
            outputs = tf.concat(outputs, axis=1, name='concat-output')
            dims = [-1, self.numSubinstance, self.numTimeSteps, self.numHidden]
            output = tf.reshape(outputs, dims, name='bag-output')

        GRUVars = cell.variables
        self.varList.extend(GRUVars)
        if self.useDropout:
            self.keep_prob = keep_prob
        self.output = output
        return self.output

    def _restoreBaseGraph(self, graph, **kwargs):
        assert self.graphCreated is False
        assert self.graph is not None
        scope = self._scope
        if self.useDropout:
            self.keep_prob = graph.get_tensor_by_name(scope + 'keep-prob:0')
        self.output = graph.get_tensor_by_name(scope + 'bag-output:0')
        kernel1 = graph.get_tensor_by_name("rnn/EMI-GRU-Cell/gates/kernel:0")
        bias1 = graph.get_tensor_by_name("rnn/EMI-GRU-Cell/gates/bias:0")
        kernel2 = graph.get_tensor_by_name(
            "rnn/EMI-GRU-Cell/candidate/kernel:0")
        bias2 = graph.get_tensor_by_name("rnn/EMI-GRU-Cell/candidate/bias:0")
        assert len(self.varList) == 0
        self.varList = [kernel1, bias1, kernel2, bias2]

    def getModelParams(self):
        '''
        Returns the GRU kernel and bias tensors.
        returns [kernel1, bias1, kernel2, bias2]
        '''
        assert self.graphCreated is True, "Graph is not created"
        assert len(self.varList) == 4
        return self.varList

    def addBaseAssignOps(self, graph, initVarList, **kwargs):
        '''
        Adds Tensorflow assignment operations to all of the model tensors.
        These operations can then be used to initialize these tensors from
        numpy matrices by running these operators

        initVarList: A list of numpy matrices that will be used for
            initialization by the assignment operation. For EMI_GRU, this
            should be list of numpy matrices corresponding to  [kernel1, bias1,
            kernel2, bias2]
        '''
        assert initVarList is not None
        assert len(initVarList) == 2
        kernel1_ = graph.get_tensor_by_name("rnn/EMI-GRU-Cell/gates/kernel:0")
        bias1_ = graph.get_tensor_by_name("rnn/EMI-GRU-Cell/gates/bias:0")
        kernel2_ = graph.get_tensor_by_name(
            "rnn/EMI-GRU-Cell/candidate/kernel:0")
        bias2_ = graph.get_tensor_by_name("rnn/EMI-GRU-Cell/candidate/bias:0")
        kernel1, bias1, kernel2, bias2 = initVarList[
            0], initVarList[1], initVarList[2], initVarList[3]
        kernel1_op = tf.assign(kernel1_, kernel1)
        bias1_op = tf.assign(bias1_, bias1)
        kernel2_op = tf.assign(kernel2_, kernel2)
        bias2_op = tf.assign(bias2_, bias2)
        self.assignOps.extend([kernel1_op, bias1_op, kernel2_op, bias2_op])


class EMI_FastRNN(EMI_RNN):

    def __init__(self, numSubinstance, numHidden, numTimeSteps,
                 numFeats, graph=None, useDropout=False,
                 update_non_linearity="tanh", wRank=None,
                 uRank=None, alphaInit=-3.0, betaInit=3.0):
        '''
        EMI-RNN using FastRNN cell. The architecture consists of a single
        FastRNN layer followed by a secondary classifier. The secondary
        classifier is not defined as part of this module and is left for the
        user to define, through the redefinition of the '_createExtendedGraph'
        and '_restoreExtendedGraph' methods.

        This class supports restoring from a meta-graph. Provide the restored
        graph as value to the graph keyword to enable this behaviour.

        numSubinstance: Number of sub-instance.
        numHidden: The dimension of the hidden state.
        numTimeSteps: The number of time steps of the RNN.
        numFeats: The feature vector dimension for each time step.
        graph: A restored metagraph. Provide a graph if restoring form a meta
            graph is required.
        useDropout: Set to True if a dropout layer is to be added
            between inputs and outputs to the RNN.
        update_non_linearity, wRank, uRank, _alphaInit, betaInit:
            These are FastRNN parameters. Please refer to FastRNN documentation
            for more information.
        '''
        self.numHidden = numHidden
        self.numTimeSteps = numTimeSteps
        self.numFeats = numFeats
        self.useDropout = useDropout
        self.numSubinstance = numSubinstance
        self.graph = graph
        self.update_non_linearity = update_non_linearity
        self.wRank = wRank
        self.uRank = uRank
        self.alphaInit = alphaInit
        self.betaInit = betaInit
        self.graphCreated = False
        # Restore or initialize
        self.keep_prob = None
        self.varList = []
        self.output = None
        self.assignOps = []
        # Internal
        self._scope = 'EMI/FastRNN/'

    def _createBaseGraph(self, X, **kwargs):
        assert self.graphCreated is False
        msg = 'X should be of form [-1, numSubinstance, numTimeSteps,'
        msg += ' numFeatures]'
        assert X.get_shape().ndims == 4, msg
        assert X.shape[1] == self.numSubinstance
        assert X.shape[2] == self.numTimeSteps
        assert X.shape[3] == self.numFeats
        # Reshape into 3D suself.h that the first dimension is -1 *
        # numSubinstance where each numSubinstance segment corresponds to one
        # bag then shape it back in into 4D
        scope = self._scope
        keep_prob = None
        with tf.name_scope(scope):
            x = tf.reshape(X, [-1, self.numTimeSteps, self.numFeats])
            x = tf.unstack(x, num=self.numTimeSteps, axis=1)
            # Get the FastRNN output
            cell = FastRNNCell(self.numHidden, self.update_non_linearity,
                               self.wRank, self.uRank, self.alphaInit,
                               self.betaInit, name='EMI-FastRNN-Cell')
            wrapped_cell = cell
            if self.useDropout is True:
                keep_prob = tf.placeholder(dtype=tf.float32, name='keep-prob')
                wrapped_cell = tf.contrib.rnn.DropoutWrapper(cell,
                                                             input_keep_prob=keep_prob,
                                                             output_keep_prob=keep_prob)
            outputs__, states = tf.nn.static_rnn(wrapped_cell, x,
                                                 dtype=tf.float32)
            outputs = []
            for output in outputs__:
                outputs.append(tf.expand_dims(output, axis=1))
            # Convert back to bag form
            outputs = tf.concat(outputs, axis=1, name='concat-output')
            dims = [-1, self.numSubinstance, self.numTimeSteps, self.numHidden]
            output = tf.reshape(outputs, dims, name='bag-output')

        FastRNNVars = cell.variables
        self.varList.extend(FastRNNVars)
        if self.useDropout:
            self.keep_prob = keep_prob
        self.output = output
        return self.output

    def _restoreBaseGraph(self, graph, **kwargs):
        assert self.graphCreated is False
        assert self.graph is not None
        scope = self._scope
        if self.useDropout:
            self.keep_prob = graph.get_tensor_by_name(scope + 'keep-prob:0')
        self.output = graph.get_tensor_by_name(scope + 'bag-output:0')

        assert len(self.varList) == 0
        if self.wRank is None:
            W = graph.get_tensor_by_name(
                "rnn/fast_rnn_cell/EMI-FastRNN-Cell/FastRNNcell/W:0")
            self.varList = [W]
        else:
            W1 = graph.get_tensor_by_name(
                "rnn/fast_rnn_cell/EMI-FastRNN-Cell/FastRNNcell/W1:0")
            W2 = graph.get_tensor_by_name(
                "rnn/fast_rnn_cell/EMI-FastRNN-Cell/FastRNNcell/W2:0")
            self.varList = [W1, W2]

        if self.uRank is None:
            U = graph.get_tensor_by_name(
                "rnn/fast_rnn_cell/EMI-FastRNN-Cell/FastRNNcell/U:0")
            self.varList.extend([U])
        else:
            U1 = graph.get_tensor_by_name(
                "rnn/fast_rnn_cell/EMI-FastRNN-Cell/FastRNNcell/U1:0")
            U2 = graph.get_tensor_by_name(
                "rnn/fast_rnn_cell/EMI-FastRNN-Cell/FastRNNcell/U2:0")
            self.varList.extend([U1, U2])

        alpha = graph.get_tensor_by_name(
            "rnn/fast_rnn_cell/EMI-FastRNN-Cell/FastRNNcell/alpha:0")
        beta = graph.get_tensor_by_name(
            "rnn/fast_rnn_cell/EMI-FastRNN-Cell/FastRNNcell/beta:0")
        bias = graph.get_tensor_by_name(
            "rnn/fast_rnn_cell/EMI-FastRNN-Cell/FastRNNcell/B_h:0")
        self.varList.extend([alpha, beta, bias])

    def getModelParams(self):
        '''
        Returns the FastRNN model tensors.
        In the order of  [W(W1, W2), U(U1,U2), alpha, beta, B_h]
        () implies that the matrix can be replaced with the matrices inside.
        '''
        assert self.graphCreated is True, "Graph is not created"
        return self.varList

    def addBaseAssignOps(self, graph, initVarList, **kwargs):
        '''
        Adds Tensorflow assignment operations to all of the model tensors.
        These operations can then be used to initialize these tensors from
        numpy matrices by running these operators

        initVarList: A list of numpy matrices that will be used for
            initialization by the assignment operation. For EMI_FastRNN, this
            should be list of numpy matrices corresponding to  [W(W1, W2),
            U(U1,U2), alpha, beta, B_h]
        '''
        assert initVarList is not None
        index = 0
        if self.wRank is None:
            W_ = graph.get_tensor_by_name(
                "rnn/fast_rnn_cell/EMI-FastRNN-Cell/FastRNNcell/W:0")
            W = initVarList[0]
            w_op = tf.assign(W_, W)
            self.assignOps.extend([w_op])
            index += 1
        else:
            W1_ = graph.get_tensor_by_name(
                "rnn/fast_rnn_cell/EMI-FastRNN-Cell/FastRNNcell/W1:0")
            W2_ = graph.get_tensor_by_name(
                "rnn/fast_rnn_cell/EMI-FastRNN-Cell/FastRNNcell/W2:0")
            W1, W2 = initVarList[0], initVarList[1]
            w1_op = tf.assign(W1_, W1)
            w2_op = tf.assign(W2_, W2)
            self.assignOps.extend([w1_op, w2_op])
            index += 2

        if self.uRank is None:
            U_ = graph.get_tensor_by_name(
                "rnn/fast_rnn_cell/EMI-FastRNN-Cell/FastRNNcell/U:0")
            U = initVarList[index]
            u_op = tf.assign(U_, U)
            self.assignOps.extend([u_op])
            index += 1
        else:
            U1_ = graph.get_tensor_by_name(
                "rnn/fast_rnn_cell/EMI-FastRNN-Cell/FastRNNcell/U1:0")
            U2_ = graph.get_tensor_by_name(
                "rnn/fast_rnn_cell/EMI-FastRNN-Cell/FastRNNcell/U2:0")
            U1, U2 = initVarList[index], initVarList[index + 1]
            u1_op = tf.assign(U1_, U1)
            u2_op = tf.assign(U2_, U2)
            self.assignOps.extend([u1_op, u2_op])
            index += 2

        alpha_ = graph.get_tensor_by_name(
            "rnn/fast_rnn_cell/EMI-FastRNN-Cell/FastRNNcell/alpha:0")
        beta_ = graph.get_tensor_by_name(
            "rnn/fast_rnn_cell/EMI-FastRNN-Cell/FastRNNcell/beta:0")
        bias_ = graph.get_tensor_by_name(
            "rnn/fast_rnn_cell/EMI-FastRNN-Cell/FastRNNcell/B_h:0")

        alpha, beta, bias = initVarList[index], initVarList[
            index + 1], initVarList[index + 2]
        alpha_op = tf.assign(alpha_, alpha)
        beta_op = tf.assign(beta_, beta)
        bias_op = tf.assign(bias_, bias)

        self.assignOps.extend([alpha_op, beta_op, bias_op])


class EMI_UGRNN(EMI_RNN):

    def __init__(self, numSubinstance, numHidden, numTimeSteps,
                 numFeats, graph=None, forgetBias=1.0, useDropout=False):
        '''
        EMI-RNN using UGRNN cell. The architecture consists of a single UGRNN
        layer followed by a secondary classifier. The secondary classifier is
        not defined as part of this module and is left for the user to define,
        through the redefinition of the '_createExtendedGraph' and
        '_restoreExtendedGraph' methods.

        This class supports restoring from a meta-graph. Provide the restored
        graph as value to the graph keyword to enable this behaviour.

        numSubinstance: Number of sub-instance.
        numHidden: The dimension of the hidden state.
        numTimeSteps: The number of time steps of the RNN.
        numFeats: The feature vector dimension for each time step.
        graph: A restored metagraph. Provide a graph if restoring form a meta
            graph is required.
        forgetBias: Bias for the forget gate of the UGRNN.
        useDropout: Set to True if a dropout layer is to be added between
            inputs and outputs to the RNN.
        '''
        self.numHidden = numHidden
        self.numTimeSteps = numTimeSteps
        self.numFeats = numFeats
        self.useDropout = useDropout
        self.forgetBias = forgetBias
        self.numSubinstance = numSubinstance
        self.graph = graph
        self.graphCreated = False
        # Restore or initialize
        self.keep_prob = None
        self.varList = []
        self.output = None
        self.assignOps = []
        # Internal
        self._scope = 'EMI/UGRNN/'

    def _createBaseGraph(self, X, **kwargs):
        assert self.graphCreated is False
        msg = 'X should be of form [-1, numSubinstance, numTimeSteps, numFeatures]'
        assert X.get_shape().ndims == 4, msg
        assert X.shape[1] == self.numSubinstance
        assert X.shape[2] == self.numTimeSteps
        assert X.shape[3] == self.numFeats
        # Reshape into 3D such that the first dimension is -1 * numSubinstance
        # where each numSubinstance segment corresponds to one bag
        # then shape it back in into 4D
        scope = self._scope
        keep_prob = None
        with tf.name_scope(scope):
            x = tf.reshape(X, [-1, self.numTimeSteps, self.numFeats])
            x = tf.unstack(x, num=self.numTimeSteps, axis=1)
            # Get the UGRNN output
            cell = tf.contrib.rnn.UGRNNCell(self.numHidden,
                                            forget_bias=self.forgetBias)
            wrapped_cell = cell
            if self.useDropout is True:
                keep_prob = tf.placeholder(dtype=tf.float32, name='keep-prob')
                wrapped_cell = tf.contrib.rnn.DropoutWrapper(cell,
                                                             input_keep_prob=keep_prob,
                                                             output_keep_prob=keep_prob)
            outputs__, states = tf.nn.static_rnn(
                wrapped_cell, x, dtype=tf.float32)
            outputs = []
            for output in outputs__:
                outputs.append(tf.expand_dims(output, axis=1))
            # Convert back to bag form
            outputs = tf.concat(outputs, axis=1, name='concat-output')
            dims = [-1, self.numSubinstance, self.numTimeSteps, self.numHidden]
            output = tf.reshape(outputs, dims, name='bag-output')

        UGRNNVars = cell.variables
        self.varList.extend(UGRNNVars)
        if self.useDropout:
            self.keep_prob = keep_prob
        self.output = output
        return self.output

    def _restoreBaseGraph(self, graph, **kwargs):
        assert self.graphCreated is False
        assert self.graph is not None
        scope = self._scope
        if self.useDropout:
            self.keep_prob = graph.get_tensor_by_name(scope + 'keep-prob:0')
        self.output = graph.get_tensor_by_name(scope + 'bag-output:0')
        kernel = graph.get_tensor_by_name("rnn/ugrnn_cell/kernel:0")
        bias = graph.get_tensor_by_name("rnn/ugrnn_cell/bias:0")
        assert len(self.varList) == 0
        self.varList = [kernel, bias]

    def getModelParams(self):
        '''
        Returns the FastRRNN model tensors.
        returns [kernel, bias]
        '''
        assert self.graphCreated is True, "Graph is not created"
        assert len(self.varList) == 2
        return self.varList

    def addBaseAssignOps(self, graph, initVarList, **kwargs):
        '''
        Adds Tensorflow assignment operations to all of the model tensors.
        These operations can then be used to initialize these tensors from
        numpy matrices by running these operators

        initVarList: A list of numpy matrices that will be used for
            initialization by the assignment operation. For EMI_UGRNN, this
            should be list of numpy matrices corresponding to  [kernel, bias]
        '''
        assert initVarList is not None
        assert len(initVarList) == 2
        k_ = graph.get_tensor_by_name('rnn/ugrnn_cell/kernel:0')
        b_ = graph.get_tensor_by_name('rnn/ugrnn_cell/bias:0')
        kernel, bias = initVarList[-2], initVarList[-1]
        k_op = tf.assign(k_, kernel)
        b_op = tf.assign(b_, bias)
        self.assignOps.extend([k_op, b_op])


class EMI_FastGRNN(EMI_RNN):

    def __init__(self, numSubinstance, numHidden, numTimeSteps, numFeats,
                 graph=None, useDropout=False, gate_non_linearity="sigmoid",
                 update_non_linearity="tanh", wRank=None, uRank=None,
                 zetaInit=1.0, nuInit=-4.0):
        '''
        EMI-RNN using FastGRNN cell. The architecture consists of a single
        FastGRNN layer followed by a secondary classifier. The secondary
        classifier is not defined as part of this module and is left for the
        user to define, through the redefinition of the '_createExtendedGraph'
        and '_restoreExtendedGraph' methods.

        This class supports restoring from a meta-graph. Provide the restored
        graph as value to the graph keyword to enable this behaviour.

        numSubinstance: Number of sub-instance.
        numHidden: The dimension of the hidden state.
        numTimeSteps: The number of time steps of the RNN.
        numFeats: The feature vector dimension for each time step.
        graph: A restored metagraph. Provide a graph if restoring form a meta
            graph is required.
        useDropout: Set to True if a dropout layer is to be added
            between inputs and outputs to the RNN.

        gate_non_linearity, update_non_linearity, wRank, uRank, zetaInit,
        nuInit:
            These are FastGRNN parameters. Please refer to FastGRNN documentation
            for more information.
        '''
        self.numHidden = numHidden
        self.numTimeSteps = numTimeSteps
        self.numFeats = numFeats
        self.useDropout = useDropout
        self.numSubinstance = numSubinstance
        self.graph = graph

        self.gate_non_linearity = gate_non_linearity
        self.update_non_linearity = update_non_linearity
        self.wRank = wRank
        self.uRank = uRank
        self.zetaInit = zetaInit
        self.nuInit = nuInit

        self.graphCreated = False
        # Restore or initialize
        self.keep_prob = None
        self.varList = []
        self.output = None
        self.assignOps = []
        # Internal
        self._scope = 'EMI/FastGRNN/'

    def _createBaseGraph(self, X, **kwargs):
        assert self.graphCreated is False
        msg = 'X should be of form [-1, numSubinstance, numTimeSteps, numFeatures]'
        assert X.get_shape().ndims == 4, msg
        assert X.shape[1] == self.numSubinstance
        assert X.shape[2] == self.numTimeSteps
        assert X.shape[3] == self.numFeats
        # Reshape into 3D suself.h that the first dimension is -1 * numSubinstance
        # where each numSubinstance segment corresponds to one bag
        # then shape it back in into 4D
        scope = self._scope
        keep_prob = None
        with tf.name_scope(scope):
            x = tf.reshape(X, [-1, self.numTimeSteps, self.numFeats])
            x = tf.unstack(x, num=self.numTimeSteps, axis=1)
            # Get the FastGRNN output
            # cell = FastGRNNCell(self.numHidden, self.gate_non_linearity,
            #                     self.update_non_linearity, self.wRank,
            #                     self.uRank, self.zetaInit, self.nuInit,
            #                     name='EMI-FastGRNN-Cell')
            # wrapped_cell = cell
            # if self.useDropout is True:
            #     keep_prob = tf.placeholder(dtype=tf.float32, name='keep-prob')
            #     wrapped_cell = tf.contrib.rnn.DropoutWrapper(cell,
            #                                                  input_keep_prob=keep_prob,
            #                                                  output_keep_prob=keep_prob)
            # outputs__, states = tf.keras.layers.LSTM(
            #     wrapped_cell, x, dtype=tf.float32)
            # outputs = []
            # for output in outputs__:
            #     outputs.append(tf.expand_dims(output, axis=1))
            # # Convert back to bag form
            # outputs = tf.concat(outputs, axis=1, name='concat-output')
            # dims = [-1, self.numSubinstance, self.numTimeSteps, self.numHidden]
            # output = tf.reshape(outputs, dims, name='bag-output')
            # Get the FastGRNN output
            cell = FastGRNNCell(self.numHidden, self.gate_non_linearity,
                    self.update_non_linearity, self.wRank,
                    self.uRank, self.zetaInit, self.nuInit,
                    name='EMI-FastGRNN-Cell')
            wrapped_cell = cell
            if self.useDropout is True:
                keep_prob = tf.placeholder(dtype=tf.float32, name='keep-prob')
                wrapped_cell = tf.contrib.rnn.DropoutWrapper(cell,
                                                 input_keep_prob=keep_prob,
                                                 output_keep_prob=keep_prob)
            outputs__, states = tf.compat.v1.nn.static_rnn(wrapped_cell, x, dtype=tf.float32)
            outputs = []
            for output in outputs__:
                outputs.append(tf.expand_dims(output, axis=1))
            # Convert back to bag form
            outputs = tf.concat(outputs, axis=1, name='concat-output')
            dims = [-1, self.numSubinstance, self.numTimeSteps, self.numHidden]
            output = tf.reshape(outputs, dims, name='bag-output')


        FastGRNNVars = cell.variables
        self.varList.extend(FastGRNNVars)
        if self.useDropout:
            self.keep_prob = keep_prob
        self.output = output
        return self.output

    def _restoreBaseGraph(self, graph, **kwargs):
        assert self.graphCreated is False
        assert self.graph is not None
        scope = self._scope
        if self.useDropout:
            self.keep_prob = graph.get_tensor_by_name(scope + 'keep-prob:0')
        self.output = graph.get_tensor_by_name(scope + 'bag-output:0')

        assert len(self.varList) == 0
        if self.wRank is None:
            W = graph.get_tensor_by_name(
                "rnn/fast_grnn_cell/EMI-FastGRNN-Cell/FastGRNNcell/W:0")
            self.varList = [W]
        else:
            W1 = graph.get_tensor_by_name(
                "rnn/fast_grnn_cell/EMI-FastGRNN-Cell/FastGRNNcell/W1:0")
            W2 = graph.get_tensor_by_name(
                "rnn/fast_grnn_cell/EMI-FastGRNN-Cell/FastGRNNcell/W2:0")
            self.varList = [W1, W2]

        if self.uRank is None:
            U = graph.get_tensor_by_name(
                "rnn/fast_grnn_cell/EMI-FastGRNN-Cell/FastGRNNcell/U:0")
            self.varList.extend([U])
        else:
            U1 = graph.get_tensor_by_name(
                "rnn/fast_grnn_cell/EMI-FastGRNN-Cell/FastGRNNcell/U1:0")
            U2 = graph.get_tensor_by_name(
                "rnn/fast_grnn_cell/EMI-FastGRNN-Cell/FastGRNNcell/U2:0")
            self.varList.extend([U1, U2])

        zeta = graph.get_tensor_by_name(
            "rnn/fast_grnn_cell/EMI-FastGRNN-Cell/FastGRNNcell/zeta:0")
        nu = graph.get_tensor_by_name(
            "rnn/fast_grnn_cell/EMI-FastGRNN-Cell/FastGRNNcell/nu:0")
        gate_bias = graph.get_tensor_by_name(
            "rnn/fast_grnn_cell/EMI-FastGRNN-Cell/FastGRNNcell/B_g:0")
        update_bias = graph.get_tensor_by_name(
            "rnn/fast_grnn_cell/EMI-FastGRNN-Cell/FastGRNNcell/B_h:0")
        self.varList.extend([zeta, nu, gate_bias, update_bias])

    def getModelParams(self):
        '''
        Returns the FastGRNN model tensors.
        In the order of  [W(W1, W2), U(U1,U2), zeta, nu, B_g, B_h]
        () implies that the matrix can be replaced with the matrices inside.
        '''
        assert self.graphCreated is True, "Graph is not created"
        return self.varList

    def addBaseAssignOps(self, graph, initVarList, **kwargs):
        '''
        Adds Tensorflow assignment operations to all of the model tensors.
        These operations can then be used to initialize these tensors from
        numpy matrices by running these operators

        initVarList: A list of numpy matrices that will be used for
            initialization by the assignment operation. For EMI_FastGRNN, this
            should be list of numpy matrices corresponding to  [W(W1, W2),
            U(U1,U2), zeta, nu, B_g, B_h]
        '''
        assert initVarList is not None
        index = 0
        if self.wRank is None:
            W_ = graph.get_tensor_by_name(
                "rnn/fast_grnn_cell/EMI-FastGRNN-Cell/FastGRNNcell/W:0")
            W = initVarList[0]
            w_op = tf.assign(W_, W)
            self.assignOps.extend([w_op])
            index += 1
        else:
            W1_ = graph.get_tensor_by_name(
                "rnn/fast_grnn_cell/EMI-FastGRNN-Cell/FastGRNNcell/W1:0")
            W2_ = graph.get_tensor_by_name(
                "rnn/fast_grnn_cell/EMI-FastGRNN-Cell/FastGRNNcell/W2:0")
            W1, W2 = initVarList[0], initVarList[1]
            w1_op = tf.assign(W1_, W1)
            w2_op = tf.assign(W2_, W2)
            self.assignOps.extend([w1_op, w2_op])
            index += 2

        if self.uRank is None:
            U_ = graph.get_tensor_by_name(
                "rnn/fast_grnn_cell/EMI-FastGRNN-Cell/FastGRNNcell/U:0")
            U = initVarList[index]
            u_op = tf.assign(U_, U)
            self.assignOps.extend([u_op])
            index += 1
        else:
            U1_ = graph.get_tensor_by_name(
                "rnn/fast_grnn_cell/EMI-FastGRNN-Cell/FastGRNNcell/U1:0")
            U2_ = graph.get_tensor_by_name(
                "rnn/fast_grnn_cell/EMI-FastGRNN-Cell/FastGRNNcell/U2:0")
            U1, U2 = initVarList[index], initVarList[index + 1]
            u1_op = tf.assign(U1_, U1)
            u2_op = tf.assign(U2_, U2)
            self.assignOps.extend([u1_op, u2_op])
            index += 2

        zeta_ = graph.get_tensor_by_name(
            "rnn/fast_grnn_cell/EMI-FastGRNN-Cell/FastGRNNcell/zeta:0")
        nu_ = graph.get_tensor_by_name(
            "rnn/fast_grnn_cell/EMI-FastGRNN-Cell/FastGRNNcell/nu:0")
        gate_bias_ = graph.get_tensor_by_name(
            "rnn/fast_grnn_cell/EMI-FastGRNN-Cell/FastGRNNcell/B_g:0")
        update_bias_ = graph.get_tensor_by_name(
            "rnn/fast_grnn_cell/EMI-FastGRNN-Cell/FastGRNNcell/B_h:0")

        zeta, nu, gate_bias, update_bias = initVarList[index], initVarList[
            index + 1], initVarList[index + 2], initVarList[index + 3]
        zeta_op = tf.assign(zeta_, zeta)
        nu_op = tf.assign(nu_, nu)
        gate_bias_op = tf.assign(gate_bias_, gate_bias)
        update_bias_op = tf.assign(update_bias_, update_bias)

        self.assignOps.extend([zeta_op, nu_op, gate_bias_op, update_bias_op])


In [11]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT license.

from __future__ import print_function
import tensorflow as tf
import numpy as np
import sys
import edgeml.tf.utils as utils
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


class EMI_Trainer:
    def __init__(self, numTimeSteps, numOutput, graph=None,
                 stepSize=0.001, lossType='l2', optimizer='Adam',
                 automode=True):
        '''
        The EMI-RNN trainer. This classes attaches loss functions and training
        operations to the forward EMI-RNN graph. Currently, simple softmax loss
        and l2 loss are supported on the outputs. For optimizers, only ADAM
        optimizer is available.

        numTimesteps: Number of time steps of the RNN model
        numOutput: Number of output classes
        graph: This module supports restoring from a meta graph. Provide the
            meta graph as an argument to enable this behaviour.
        lossType: A valid loss type string in ['l2', 'xentropy'].
        optimizer: A valid optimizer string in ['Adam'].
        automode: Disable or enable the automode behaviour.
        This module takes care of all of the training procedure automatically,
        and the default behaviour is suitable for most cases. In certain cases
        though, the user would want to change certain aspects of the graph;
        specifically, he would want to change to loss operation by, say, adding
        regularization terms for the model matrices. To enable this behaviour,
        the user can perform the following steps:
            1. Disable automode. That is, when initializing, set automode=False
            2. After the __call__ method has been invoked to create the loss
            operation, the user can access the self.lossOp attribute and modify
            it by adding regularization or other terms.
            3. After the modification has been performed, the user needs to
            call the `createOpCollections()` method so that the newly edited
            operations can be added to Tensorflow collections. This helps in

        HELP_WANTED: Automode is more of a hack than a systematic way of
        supporting multiple loss functions/ optimizers. One way of
        accomplishing this would be to make __createTrainOp and __createLossOp
        methods protected or public, and having users override these.
        Alternatively, we can change the structure to incorporate the
        _createExtendedGraph and _restoreExtendedGraph operations used in
        EMI-LSTM and so forth.
        '''
        self.numTimeSteps = numTimeSteps
        self.numOutput = numOutput
        self.graph = graph
        self.stepSize = stepSize
        self.lossType = lossType
        self.optimizer = optimizer
        self.automode = automode
        self.__validInit = False
        self.graphCreated = False
        # Operations to be restored
        self.lossOp = None
        self.trainOp = None
        self.softmaxPredictions = None
        self.accTilda = None
        self.equalTilda = None
        self.lossIndicatorTensor = None
        self.lossIndicatorPlaceholder = None
        self.lossIndicatorAssignOp = None
        # Input validation
        self.supportedLosses = ['xentropy', 'l2']
        self.supportedOptimizers = ['Adam']
        assert lossType in self.supportedLosses
        assert optimizer in self.supportedOptimizers
        # Internal
        self.scope = 'EMI/Trainer/'

    def __validateInit(self, predicted, target):
        msg = 'Predicted/Target tensors have incorrect dimension'
        assert len(predicted.shape) == 4, msg
        assert predicted.shape[3] == self.numOutput, msg
        assert predicted.shape[2] == self.numTimeSteps, msg
        assert predicted.shape[1] == target.shape[1], msg
        assert len(target.shape) == 3
        assert target.shape[2] == self.numOutput
        self.__validInit = True

    def __call__(self, predicted, target):
        '''
        Constructs the loss and train operations. If already created, returns
        the created operators.

        predicted: The prediction scores outputed from the forward computation
            graph. Expects a 4 dimensional tensor with shape [-1,
            numSubinstance, numTimeSteps, numClass].
        target: The target labels in one hot-encoding. Expects [-1,
            numSubinstance, numClass]
        '''
        if self.graphCreated is True:
            # TODO: These statements are redundant after self.validInit call
            # A simple check to self.__validInit should suffice. Test this.
            assert self.lossOp is not None
            assert self.trainOp is not None
            return self.lossOp, self.trainOp
        self.__validateInit(predicted, target)
        assert self.__validInit is True
        if self.graph is None:
            self._createGraph(predicted, target)
        else:
            self._restoreGraph(predicted, target)
        assert self.graphCreated == True
        return self.lossOp, self.trainOp

    def __transformY(self, target):
        '''
        Because we need output from each step and not just the last step.
        Currently we just tile the target to each step. This method can be
        exteneded/overridden to allow more complex behaviours
        '''
        with tf.name_scope(self.scope):
            A_ = tf.expand_dims(target, axis=2)
            A__ = tf.tile(A_, [1, 1, self.numTimeSteps, 1])
        return A__

    def __createLossOp(self, predicted, target):
        assert self.__validInit is True, 'Initialization failure'
        with tf.name_scope(self.scope):
            # Loss indicator tensor
            li = np.zeros([self.numTimeSteps, self.numOutput])
            li[-1, :] = 1
            liTensor = tf.Variable(li.astype('float32'),
                                   name='loss-indicator',
                                   trainable=False)
            name='loss-indicator-placeholder'
            liPlaceholder = tf.compat.v1.placeholder(tf.float32, shape=(None, None), name='loss-indicator-placeholder')


            liAssignOp = tf.compat.v1.assign(liTensor, liPlaceholder,
                                   name='loss-indicator-assign-op')
            self.lossIndicatorTensor = liTensor
            self.lossIndicatorPlaceholder = liPlaceholder
            self.lossIndicatorAssignOp = liAssignOp
            # predicted of dim [-1, numSubinstance, numTimeSteps, numOutput]
            dims = [-1, self.numTimeSteps, self.numOutput]
            logits__ = tf.reshape(predicted, dims)
            labels__ = tf.reshape(target, dims)
            diff = (logits__ - labels__)
            diff = tf.multiply(self.lossIndicatorTensor, diff)
            # take loss only for the timesteps indicated by lossIndicator for softmax
            logits__ = tf.multiply(self.lossIndicatorTensor, logits__)
            labels__ = tf.multiply(self.lossIndicatorTensor, labels__)
            logits__ = tf.reshape(logits__, [-1, self.numOutput])
            labels__ = tf.reshape(labels__, [-1, self.numOutput])
            # Regular softmax
            if self.lossType == 'xentropy':
                softmax1 = tf.nn.softmax_cross_entropy_with_logits(labels=labels__,logits=logits__)
                lossOp = tf.reduce_mean(softmax1, name='xentropy-loss')
            elif self.lossType == 'l2':
                lossOp = tf.nn.l2_loss(diff, name='l2-loss')
        return lossOp
    def __createTrainOp(self):
        with tf.name_scope(self.scope):
            optimizer = tf.keras.optimizers.Adam(self.stepSize)
            trainable_variables = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.TRAINABLE_VARIABLES, scope=self.scope)
            train_op = optimizer.minimize(self.lossOp, var_list=trainable_variables)
        return train_op





    def _createGraph(self, predicted, target):
        target = self.__transformY(target)
        assert self.__validInit is True
        with tf.name_scope(self.scope):
            self.softmaxPredictions = tf.nn.softmax(predicted, axis=3,
                                                    name='softmaxed-prediction')
            pred = self.softmaxPredictions[:, :, -1, :]
            actu = target[:, :, -1, :]
            resPred = tf.reshape(pred, [-1, self.numOutput])
            resActu = tf.reshape(actu, [-1, self.numOutput])
            maxPred = tf.argmax(resPred, axis=1)
            maxActu = tf.argmax(resActu, axis=1)
            equal = tf.equal(maxPred, maxActu)
            self.equalTilda = tf.cast(equal, tf.float32, name='equal-tilda')
            self.accTilda = tf.reduce_mean(self.equalTilda, name='acc-tilda')

        self.lossOp = self.__createLossOp(predicted, target)
        self.trainOp = self.__createTrainOp()
        if self.automode:
            self.createOpCollections()
        self.graphCreated = True

    def _restoreGraph(self, predicted, target):
        assert self.graphCreated is False
        scope = self.scope
        graph = self.graph
        self.trainOp = tf.get_collection('EMI-train-op')
        self.lossOp = tf.get_collection('EMI-loss-op')
        msg0 = 'Operator or tensor not found'
        msg1 = 'Multiple tensors with the same name in the graph. Are you not'
        msg1 +=' resetting your graph?'
        assert len(self.trainOp) != 0, msg0
        assert len(self.lossOp) != 0, msg0
        assert len(self.trainOp) == 1, msg1
        assert len(self.lossOp) == 1, msg1
        self.trainOp = self.trainOp[0]
        self.lossOp = self.lossOp[0]
        self.lossIndicatorTensor = graph.get_tensor_by_name(scope +
                                                            'loss-indicator:0')
        name = 'loss-indicator-placeholder:0'
        self.lossIndicatorPlaceholder = graph.get_tensor_by_name(scope + name)
        name = 'loss-indicator-assign-op:0'
        self.lossIndicatorAssignOp = graph.get_tensor_by_name(scope + name)
        name = scope + 'softmaxed-prediction:0'
        self.softmaxPredictions = graph.get_tensor_by_name(name)
        name = scope + 'acc-tilda:0'
        self.accTilda = graph.get_tensor_by_name(name)
        name = scope + 'equal-tilda:0'
        self.equalTilda = graph.get_tensor_by_name(name)
        self.graphCreated = True
        self.__validInit = True

    def createOpCollections(self):
        '''
        Adds the trainOp and lossOp to Tensorflow collections. This enables us
        to restore these operations from saved metagraphs.
        '''
        tf.add_to_collection('EMI-train-op', self.trainOp)
        tf.add_to_collection('EMI-loss-op', self.lossOp)

    def __echoCB(self, sess, feedDict, currentBatch, redirFile, **kwargs):
        _, loss = sess.run([self.trainOp, self.lossOp],
                                feed_dict=feedDict)
        print("\rBatch %5d Loss %2.5f" % (currentBatch, loss),
              end='', file=redirFile)

    def trainModel(self, sess, redirFile=None, echoInterval=15,
                   echoCB=None, feedDict=None, **kwargs):
        '''
        The training routine.

        sess: The Tensorflow session associated with the computation graph.
        redirFile: Output from the training routine can be redirected to a file
            on the disk. Please provide the file pointer to said file to enable
            this behaviour. Defaults to STDOUT. To disable outputs all
            together, please pass a file pointer to DEVNULL or equivalent as an
            argument.
        echoInterval: The number of batch updates between calls to echoCB.
        echoCB: This call back method is used for printing intermittent
            training stats such as validation accuracy or loss value. By default,
            it defaults to self.__echoCB. The signature of the method is,

            echoCB(self, session, feedDict, currentBatch, redirFile, **kwargs)

            Please refer to the __echoCB implementation for a simple example.
            A more complex example can be found in the EMI_Driver.
        feedDict: feedDict, that is required for the session.run() calls. Will
            be directly passed to the sess.run() calls.
        **kwargs: Additional args to echoCB.
        '''
        if echoCB is None:
            echoCB = self.__echoCB
        currentBatch = 0
        while True:
            try:
                if currentBatch % echoInterval == 0:
                    echoCB(sess, feedDict, currentBatch, redirFile, **kwargs)
                else:
                    sess.run([self.trainOp], feed_dict=feedDict)
                currentBatch += 1
            except tf.errors.OutOfRangeError:
                break

    def restoreFromGraph(self, graph):
        '''
        This method provides an alternate way of restoring
        from a saved meta graph - without having to provide the restored meta
        graph as a parameter to __init__. This is useful when, in between
        training, you want to reset the entire computation graph and reload a
        new meta graph from disk. This method allows you to attach to this
        newly loaded meta graph without having to create a new EMI_Trainer
        object. Use this method only when you want to clear/reset the existing
        computational graph.
        '''
        self.graphCreated = False
        self.lossOp = None
        self.trainOp = None
        self.lossIndicatorTensor = None
        self.softmaxPredictions = None
        self.accTilda = None
        self.graph = graph
        self.__validInit = True
        assert self.graphCreated is False
        self._restoreGraph(None, None)
        assert self.graphCreated is True


class EMI_Driver:
    def __init__(self, emiDataPipeline, emiGraph, emiTrainer,
                 max_to_keep=1000, globalStepStart=1000):
        '''
        The driver class that takes care of training an EMI RNN graph. The EMI
        RNN graph consists of three parts - a data input pipeline
        (EMI_DataPipeline), the forward computation graph (EMI-RNN) and the
        loss graph (EMI_Trainer). After the three parts of been created and
        connected, they should be passed as arguments to this module.

        Since EMI-RNN training requires careful handling of Sessions and
        graphs, these details are wrapped inside EMI_Driver. For an external
        method to access the current session, please make sure to use
        getCurrentSession() method defined in EMI_Driver. Note that this has to
        be done every time a reference to the current session is required.
        (Internally, sessions are closed and opened when new models are loaded
        from the disk, so technically, as long as no new model has been loaded
        from disk, there is no need to call getCurrentSession() again - it is
        better to be safe though).

        emiDataPipeline: An EMI_DataPipeline object.
        emiGraph: An EMI_RNN object.
        emiTrainer: An EMI_Trainer object.
        max_to_keep: Maximum number of model checkpoints to keep. Make sure
            that this is more than [number of iterations] * [number of rounds].
        globalStepStart: The global step  value is used as a key for naming
        saved meta graphs. Meta graphs and checkpoints will be named from
        globalStepStart through globalStepStart  + max_to_keep.
        '''
        self._dataPipe = emiDataPipeline
        self._emiGraph = emiGraph
        self._emiTrainer = emiTrainer
        msg = 'Have you invoked __call__()'
        assert self._dataPipe.graphCreated is True, msg
        assert self._emiGraph.graphCreated is True, msg
        assert self._emiTrainer.graphCreated is True, msg
        self.__globalStep = globalStepStart
        self.__saver = tf.train.Saver(max_to_keep=max_to_keep,
                                      save_relative_paths=True)
        self.__graphManager = utils.GraphManager()
        self.__sess = None

    def fancyEcho(self, sess, feedDict, currentBatch, redirFile,
                  numBatches=None):
        '''
        A callable that is passed as argument - echoCB - to
        EMI_Trainer.train() method.
        '''
        _, loss, acc = sess.run([self._emiTrainer.trainOp,
                                 self._emiTrainer.lossOp,
                                 self._emiTrainer.accTilda],
                                feed_dict=feedDict)
        epoch = int(currentBatch /  numBatches)
        batch = int(currentBatch % max(numBatches, 1))
        print("\rEpoch %3d Batch %5d (%5d) Loss %2.5f Acc %2.5f |" %
              (epoch, batch, currentBatch, loss, acc),
              end='', file=redirFile)

    def assignToGraph(self, initVarList):
        '''
        This method should deal with restoring the entire graph
        now'''
        raise NotImplementedError()

    def initializeSession(self, graph, reuse=False, feedDict=None):
        '''
        Initialize a new session with the computation graph provided in graph.

        graph: The computation graph needed to be used for the current session.
        reuse: If True, global_variables_initializer will not be invoked and
            the graph will retain the current tensor states/values. 
        feedDict: Not used
        '''
        sess = self.__sess
        if sess is not None:
           sess.close()
        with graph.as_default():
            sess = tf.Session()
        if reuse is False:
            with graph.as_default():
                init = tf.global_variables_initializer()
            sess.run(init)
        self.__sess = sess

    def getCurrentSession(self):
        '''
        Returns the current tf.Session()
        '''
        return self.__sess

    def setSession(self, sess):
        '''
        Sets sess as the session to be used by the driver. Experimental and not
        recommended.
        '''
        self.__sess = sess

    def runOps(self, opList, X, Y, batchSize, feedDict=None, **kwargs):
        '''
        Run tensorflow operations provided in opList on data X, Y.

        opList: A list of operations.
        X, Y: Numpy matrices of the data.
        batchSize: batch size
        feedDict: Feed dict required, if any, by the provided ops.

        returns a  list of batchwise results of sess.run(opList) on the
        provided data.
        '''
        sess = self.__sess
        if feedDict is None:
            feedDict = self.feedDictFunc(**kwargs)
        self._dataPipe.runInitializer(sess, X, Y, batchSize,
                                       numEpochs=1)
        outList = []
        while True:
            try:
                resList = sess.run(opList, feed_dict=feedDict)
                outList.append(resList)
            except tf.errors.OutOfRangeError:
                break
        return outList

    def run(self, numClasses, x_train, y_train, bag_train, x_val, y_val,
            bag_val, numIter, numRounds, batchSize, numEpochs, echoCB=None,
            redirFile=None, modelPrefix='/tmp/model', updatePolicy='top-k',
            fracEMI=0.3, lossIndicator=None, *args, **kwargs):
        '''
        Performs the EMI-RNN training routine.

        numClasses: Number of output classes.
        x_train, y_train, bag_train, x_val, y_val, bag_val: data matrices for
            test and validation sets. Please refer to the data preparation
            document for more information.
        numIter: Number of iterations. Each iteration consists of numEpochs
            passes of the data. A model check point is created after each
            iteration.
        numRounds: Number of rounds of label updates to perform. Each round
            consists of numIter iterations of numEpochs passes over the data.
        batchSize: Batch Size.
        numEpochs: Number of epochs per iteration. A model checkpoint is
            created after evey numEpochs passes over the data.
        feedDict: Feed dict for training procedure (optional).
        echoCB: The echo function (print function) that is passed to the
            EMI_Trainer.trian() method. Defaults to self.fancyEcho()
        redirFile: Provide a file pointer to redirect output to if required.
        modelPrefix: Output directory/prefix for checkpoints and metagraphs.
        updatePolicy: Supported values are 'top-k' and 'prune-ends'. Refer to
            the update policy documentation for more information.
        fracEMI: Fraction of the total rounds that use EMI-RNN loss. The
            initial (1-fracEMI) rounds will use regular MI-RNN loss. To perform
            only MI-RNN training, set this to 0.0.
        lossIndicator: NotImplemented
        *args, **kwargs: Additional arguments passed to callback methods and
            update policy methods.

        returns the updated instance level labels on the training set and a
            list of model stats after each round.
        '''
        assert self.__sess is not None, 'No sessions initialized'
        sess = self.__sess
        assert updatePolicy in ['prune-ends', 'top-k']
        if updatePolicy == 'top-k':
            print("Update policy: top-k", file=redirFile)
            updatePolicyFunc = self.__policyTopK
        else:
            print("Update policy: prune-ends", file=redirFile)
            updatePolicyFunc = self.__policyPrune

        curr_y = np.array(y_train)
        assert fracEMI >= 0
        assert fracEMI <= 1
        emiSteps = int(fracEMI * numRounds)
        emiStep = numRounds - emiSteps
        print("Training with MI-RNN loss for %d rounds" % emiStep,
              file=redirFile)
        modelStats = []
        for cround in range(numRounds):
            feedDict = self.feedDictFunc(inference=False, **kwargs)
            print("Round: %d" % cround, file=redirFile)
            if cround == emiStep:
                print("Switching to EMI-Loss function", file=redirFile)
                if lossIndicator is not None:
                    raise NotImplementedError('TODO')
                else:
                    nTs = self._emiTrainer.numTimeSteps
                    nOut = self._emiTrainer.numOutput
                    lossIndicator = np.ones([nTs, nOut])
                    sess.run(self._emiTrainer.lossIndicatorAssignOp,
                         feed_dict={self._emiTrainer.lossIndicatorPlaceholder:
                                    lossIndicator})
            valAccList, globalStepList = [], []
            # Train the best model for the current round
            for citer in range(numIter):
                self._dataPipe.runInitializer(sess, x_train, curr_y,
                                               batchSize, numEpochs)
                numBatches = int(np.ceil(len(x_train) / batchSize))
                self._emiTrainer.trainModel(sess, echoCB=self.fancyEcho,
                                             numBatches=numBatches,
                                             feedDict=feedDict,
                                             redirFile=redirFile)
                if self._emiGraph.useDropout is True:
                    ret = self.getInstancePredictions(x_val, y_val,
                                                      self.__nonEarlyInstancePrediction,
                                                      keep_prob=1.0)
                else:
                    ret = self.getInstancePredictions(x_val, y_val,
                                                      self.__nonEarlyInstancePrediction)
                predictions = ret[0]
                numSubinstance = x_val.shape[1]
                numOutput = self._emiTrainer.numOutput
                df = self.analyseModel(predictions, bag_val, numSubinstance,
                                       numOutput, silent=True)
                acc = np.max(df['acc'].values)
                print(" Val acc %2.5f | " % acc, end='', file=redirFile)
                self.__graphManager.checkpointModel(self.__saver, sess,
                                                    modelPrefix,
                                                    self.__globalStep,
                                                    redirFile=redirFile)
                valAccList.append(acc)
                globalStepList.append((modelPrefix, self.__globalStep))
                self.__globalStep += 1

            # Update y for the current round
            ## Load the best val-acc model
            argAcc = np.argmax(valAccList)
            resPrefix, resStep = globalStepList[argAcc]
            modelStats.append((cround, np.max(valAccList),
                               resPrefix, resStep))
            self.loadSavedGraphToNewSession(resPrefix, resStep, redirFile)
            sess = self.getCurrentSession()
            feedDict = self.feedDictFunc(inference=True, **kwargs)
            smxOut = self.runOps([self._emiTrainer.softmaxPredictions],
                                     x_train, y_train, batchSize, feedDict)
            smxOut= [np.array(smxOut[i][0]) for i in range(len(smxOut))]
            smxOut = np.concatenate(smxOut)[:, :, -1, :]
            newY = updatePolicyFunc(curr_y, smxOut, bag_train,
                                    numClasses, **kwargs)
            currY = newY
        return currY, modelStats

    def loadSavedGraphToNewSession(self, modelPrefix, globalStep,
                                      redirFile=None):
        self.__sess.close()
        tf.reset_default_graph()
        sess = tf.Session()
        graph = self.__graphManager.loadCheckpoint(sess, modelPrefix,
                                                   globalStep=globalStep,
                                                   redirFile=redirFile)
        # return graph
        self._dataPipe.restoreFromGraph(graph)
        self._emiGraph.restoreFromGraph(graph)
        self._emiTrainer.restoreFromGraph(graph)
        self.__sess = sess
        return graph

    def updateLabel(self, Y, policy, softmaxOut, bagLabel, numClasses, **kwargs):
        '''
        Updates the current label information based on policy and the predicted
        outputs.

        Y: numpy array of current label information.
        policy: The update policy to use. Currently supports ['top-k',
            'prune-ends']
        softmaxOut: The predicted instance level output from the soft-max
            layer.
        bagLabel: A numpy array with bag level label information.
        numClasses: Number of output classes.
        **kwargs: Additional keyword arguments to the update policy
        '''
        assert policy in ['prune-ends', 'top-k']
        if policy == 'top-k':
            updatePolicyFunc = self.__policyTopK
        else:
            updatePolicyFunc = self.__policyPrune
        Y_ = np.array(Y)
        newY = updatePolicyFunc(Y_, softmaxOut, bagLabel, numClasses, **kwargs)
        return newY

    def analyseModel(self, predictions, Y_bag, numSubinstance, numClass,
                     redirFile=None, verbose=False, silent=False):
        '''
        Some basic analysis on predictions and true labels.

        predictions: [-1, numsubinstance] is the instance level prediction.
        Y_Bag: [-1] is the bag level labels.
        numSubinstace: Number of sub-instance.
        numClass: Number of classes.
        redirFile: To redirect output to a file, provide the file pointer.

        verbose: Prints verbose data frame. Includes additionally, precision
            and recall information.

        silent: Disable output to console or file pointer.
        '''
        assert (predictions.ndim == 2)
        assert (predictions.shape[1] == numSubinstance)
        assert (Y_bag.ndim == 1)
        assert (len(Y_bag) == len(predictions))
        pholder = [0.0] * numSubinstance
        df = pd.DataFrame()
        df['len'] = np.arange(1, numSubinstance + 1)
        df['acc'] = pholder
        df['macro-fsc'] = pholder
        df['macro-pre'] = pholder
        df['macro-rec'] = pholder

        df['micro-fsc'] = pholder
        df['micro-pre'] = pholder
        df['micro-rec'] = pholder
        colList = []
        colList.append('acc')
        colList.append('macro-fsc')
        colList.append('macro-pre')
        colList.append('macro-rec')

        colList.append('micro-fsc')
        colList.append('micro-pre')
        colList.append('micro-rec')
        for i in range(0, numClass):
            pre = 'pre_%02d' % i
            rec = 'rec_%02d' % i
            df[pre] = pholder
            df[rec] = pholder
            colList.append(pre)
            colList.append(rec)

        for i in range(1, numSubinstance + 1):
            pred_ = self.getBagPredictions(predictions, numClass=numClass,
                                           minSubsequenceLen=i,
                                           redirFile = redirFile)
            correct = (pred_ == Y_bag).astype('int')
            trueAcc = np.mean(correct)
            cmatrix = utils.getConfusionMatrix(pred_, Y_bag, numClass)
            df.iloc[i-1, df.columns.get_loc('acc')] = trueAcc

            macro, micro = utils.getMacroMicroFScore(cmatrix)
            df.iloc[i-1, df.columns.get_loc('macro-fsc')] = macro
            df.iloc[i-1, df.columns.get_loc('micro-fsc')] = micro

            pre, rec = utils.getMacroPrecisionRecall(cmatrix)
            df.iloc[i-1, df.columns.get_loc('macro-pre')] = pre
            df.iloc[i-1, df.columns.get_loc('macro-rec')] = rec

            pre, rec = utils.getMicroPrecisionRecall(cmatrix)
            df.iloc[i-1, df.columns.get_loc('micro-pre')] = pre
            df.iloc[i-1, df.columns.get_loc('micro-rec')] = rec
            for j in range(numClass):
                pre, rec = utils.getPrecisionRecall(cmatrix, label=j)
                pre_ = df.columns.get_loc('pre_%02d' % j)
                rec_ = df.columns.get_loc('rec_%02d' % j)
                df.iloc[i-1, pre_ ] = pre
                df.iloc[i-1, rec_ ] = rec

        df.set_index('len')
        # Comment this line to include all columns
        colList = ['len', 'acc', 'macro-fsc', 'macro-pre', 'macro-rec']
        colList += ['micro-fsc', 'micro-pre', 'micro-rec']
        if verbose:
            for col in df.columns:
                if col not in colList:
                    colList.append(col)
        if numClass == 2:
            precisionList = df['pre_01'].values
            recallList = df['rec_01'].values
            denom = precisionList + recallList
            denom[denom == 0] = 1
            numer = 2 * precisionList * recallList
            f_ = numer / denom
            df['fscore_01'] = f_
            colList.append('fscore_01')

        df = df[colList]
        if silent is True:
            return df

        with pd.option_context('display.max_rows', 100,
                               'display.max_columns', 100,
                               'expand_frame_repr', True):
            print(df, file=redirFile)

        idx = np.argmax(df['acc'].values)
        val = np.max(df['acc'].values)
        print("Max accuracy %f at subsequencelength %d" % (val, idx + 1),
              file=redirFile)
        val = np.max(df['micro-fsc'].values)
        idx = np.argmax(df['micro-fsc'].values)
        print("Max micro-f %f at subsequencelength %d" % (val, idx + 1),
              file=redirFile)
        val = df['micro-pre'].values[idx]
        print("Micro-precision %f at subsequencelength %d" % (val, idx + 1),
              file=redirFile)
        val = df['micro-rec'].values[idx]
        print("Micro-recall %f at subsequencelength %d" % (val, idx + 1),
              file=redirFile)

        idx = np.argmax(df['macro-fsc'].values)
        val = np.max(df['macro-fsc'].values)
        print("Max macro-f %f at subsequencelength %d" % (val, idx + 1),
              file=redirFile)
        val = df['macro-pre'].values[idx]
        print("macro-precision %f at subsequencelength %d" % (val, idx + 1),
              file=redirFile)
        val = df['macro-rec'].values[idx]
        print("macro-recall %f at subsequencelength %d" % (val, idx + 1),
              file=redirFile)
        if numClass == 2 and verbose:
            idx = np.argmax(df['fscore_01'].values)
            val = np.max(df['fscore_01'].values)
            print('Max fscore %f at subsequencelength %d' % (val, idx + 1),
                  file=redirFile)
            print('Precision %f at subsequencelength %d' %
                  (df['pre_01'].values[idx], idx + 1), file=redirFile)
            print('Recall %f at subsequencelength %d' %
                  (df['rec_01'].values[idx], idx + 1), file=redirFile)
        return df

    def __nonEarlyInstancePrediction(self, instanceOut, **kwargs):
        '''
        A prediction policy used internally. No early prediction is performed
        and the class with max prob at the last step of the RNN is returned.
        '''
        assert instanceOut.ndim == 2
        retclass = np.argmax(instanceOut[-1])
        step = len(instanceOut) - 1
        return retclass, step

    def getInstancePredictions(self, x, y, earlyPolicy, batchSize=1024,
                               feedDict=None, **kwargs):

        '''
        Returns instance level predictions for data (x, y).

        Takes the softmax outputs from the joint trained model and, applies
        earlyPolicy() on each instance and returns the instance level
        prediction as well as the step at which this prediction was made.

        softmaxOut: [-1, numSubinstance, numTimeSteps, numClass]
        earlyPolicy: callable,
            def earlyPolicy(subinstacePrediction):
                subinstacePrediction: [numTimeSteps, numClass]
                ...
                return predictedClass, predictedStep

        returns: predictions, predictionStep
            predictions: [-1, numSubinstance]
            predictionStep: [-1, numSubinstance]
        '''
        opList = self._emiTrainer.softmaxPredictions
        if 'keep_prob' in kwargs:
            assert kwargs['keep_prob'] == 1, 'Keep prob should be 1.0'
        smxOut = self.runOps(opList, x, y, batchSize, feedDict=feedDict,
                             **kwargs)
        softmaxOut = np.concatenate(smxOut, axis=0)
        assert softmaxOut.ndim == 4
        numSubinstance, numTimeSteps, numClass = softmaxOut.shape[1:]
        softmaxOutFlat = np.reshape(softmaxOut, [-1, numTimeSteps, numClass])
        flatLen = len(softmaxOutFlat)
        predictions = np.zeros(flatLen)
        predictionStep = np.zeros(flatLen)
        for i, instance in enumerate(softmaxOutFlat):
            # instance is [numTimeSteps, numClass]
            assert instance.ndim == 2
            assert instance.shape[0] == numTimeSteps
            assert instance.shape[1] == numClass
            predictedClass, predictedStep = earlyPolicy(instance, **kwargs)
            predictions[i] = predictedClass
            predictionStep[i] = predictedStep
        predictions = np.reshape(predictions, [-1, numSubinstance])
        predictionStep = np.reshape(predictionStep, [-1, numSubinstance])
        return predictions, predictionStep

    def getBagPredictions(self, Y_predicted, minSubsequenceLen = 4,
                          numClass=2, redirFile = None):
        '''
        Returns bag level predictions given instance level predictions.

        A bag is considered to belong to a non-zero class if
        minSubsequenceLen is satisfied. Otherwise, it is assumed
        to belong to class 0. class 0 is negative by default. If
        minSubsequenceLen is satisfied by multiple classes, the smaller of the
        two is returned

        Y_predicted is the predicted instance level results
        [-1, numsubinstance]
        Y True is the correct instance level label
        [-1, numsubinstance]
        '''
        assert(Y_predicted.ndim == 2)
        scoreList = []
        for x in range(1, numClass):
            scores = self.__getLengthScores(Y_predicted, val=x)
            length = np.max(scores, axis=1)
            scoreList.append(length)
        scoreList = np.array(scoreList)
        scoreList = scoreList.T
        assert(scoreList.ndim == 2)
        assert(scoreList.shape[0] == Y_predicted.shape[0])
        assert(scoreList.shape[1] == numClass - 1)
        length = np.max(scoreList, axis=1)
        assert(length.ndim == 1)
        assert(length.shape[0] == Y_predicted.shape[0])
        predictionIndex = (length >= minSubsequenceLen)
        prediction = np.zeros((Y_predicted.shape[0]))
        labels = np.argmax(scoreList, axis=1) + 1
        prediction[predictionIndex] = labels[predictionIndex]
        return prediction.astype(int)

    def __getLengthScores(self, Y_predicted, val=1):
        '''
        Returns an matrix which contains the length of the longest positive
        subsequence of val ending at that index.
        Y_predicted: [-1, numSubinstance] Is the instance level class
            labels.
        '''
        scores = np.zeros(Y_predicted.shape)
        for i, bag in enumerate(Y_predicted):
            for j, instance in enumerate(bag):
                prev = 0
                if j > 0:
                    prev = scores[i, j-1]
                if instance == val:
                    scores[i, j] = prev + 1
                else:
                    scores[i, j] = 0
        return scores

    def __policyPrune(self, currentY, softmaxOut, bagLabel, numClasses,
                      minNegativeProb=0.0, updatesPerCall=3,
                      maxAllowedUpdates=3, **kwargs):
        '''
        CurrentY: [-1, numsubinstance, numClass]
        softmaxOut: [-1, numsubinstance, numClass]
        bagLabel: [-1]
        numClasses: Number of output classes
        minNegativeProb: A instance predicted as negative is labeled as
            negative iff prob. negative >= minNegativeProb
        updatesPerCall: At most number of updates to per function call
        maxAllowedUpdates: Total updates on positive bag cannot exceed
            maxAllowedUpdate.

        This policy incrementally increases the prefix/suffix of negative
        labels in currentY.  An instance is labelled as a negative if:

            1. All the instances preceding it in case of a prefix and all
            instances succeeding it in case of a continuous prefix and/or
            suffix of negative is labeled as a negative.
            2. The probability of the instance being negative > negativeProb.
            3. The instance is indeed predicted as negative (i.e. prob class 0
            is max)
            4. If the sequence length is less than maxSamples.

        All four conditions must hold. In case of a tie between instances near
        the suffix and prefix, the one with maximum probability is updated. If
        probabilities are same, then the left prefix is updated.

        CLASS 0 is assumed to be negative class
        '''
        assert currentY.ndim == 3
        assert softmaxOut.ndim == 3
        assert bagLabel.ndim == 1
        assert len(currentY) == len(softmaxOut)
        assert len(softmaxOut) == len(bagLabel)
        numSubinstance = currentY.shape[1]
        assert maxAllowedUpdates < numSubinstance
        assert softmaxOut.shape[1] == numSubinstance

        index = (bagLabel != 0)
        indexList = np.where(bagLabel)[0]
        newY = np.array(currentY)
        for i in indexList:
            currLabel = currentY[i]
            currProbabilities = softmaxOut[i]
            prevPrefix = 0
            prevSuffix = 0
            for inst in currLabel:
                if np.argmax(inst) == 0:
                    prevPrefix += 1
                else:
                    break
            for inst in reversed(currLabel):
                if np.argmax(inst) == 0:
                    prevSuffix += 1
                else:
                    break
            assert (prevPrefix + prevSuffix <= maxAllowedUpdates)
            leftIdx = int(prevPrefix)
            rightIdx = numSubinstance - int(prevSuffix) - 1
            possibleUpdates = min(updatesPerCall, maxAllowedUpdates - prevPrefix - prevSuffix)
            while (possibleUpdates > 0):
                assert leftIdx < numSubinstance
                assert leftIdx >= 0
                assert rightIdx < numSubinstance
                assert rightIdx >= 0
                leftLbl = np.argmax(currProbabilities[leftIdx])
                leftProb = np.max(currProbabilities[leftIdx])
                rightLbl = np.argmax(currProbabilities[rightIdx])
                rightProb = np.max(currProbabilities[rightIdx])
                if (leftLbl != 0 and rightLbl !=0):
                    break
                elif (leftLbl == 0 and rightLbl != 0):
                    if leftProb >= minNegativeProb:
                        newY[i, leftIdx, :] = 0
                        newY[i, leftIdx, 0] = 1
                        leftIdx += 1
                    else:
                        break
                elif (leftLbl != 0 and rightLbl == 0):
                    if rightProb >= minNegativeProb:
                        newY[i, rightIdx, :] = 0
                        newY[i, rightIdx, 0] = 1
                        rightIdx -= 1
                    else:
                        break
                elif leftProb >= rightProb:
                    if leftProb >= minNegativeProb:
                        newY[i, leftIdx, :] = 0
                        newY[i, leftIdx, 0] = 1
                        leftIdx += 1
                    else:
                        break
                elif rightProb > leftProb:
                    if rightProb >= minNegativeProb:
                        newY[i, rightIdx, :] = 0
                        newY[i, rightIdx, 0] = 1
                        rightIdx -= 1
                    else:
                        break
                possibleUpdates -= 1
        return newY

    def __policyTopK(self, currentY, softmaxOut, bagLabel, numClasses, k=1,
                     **kwargs):
        '''
        currentY: [-1, numsubinstance, numClass]
        softmaxOut: [-1, numsubinstance, numClass]
        bagLabel [-1]
        k: minimum length of continuous non-zero examples

        Algorithm:
            For each bag:
                1. Find the longest continuous subsequence of a label.
                2. If this label is the same as the bagLabel, and if the length
                of the subsequence is at least k:
                    2.1 Set the label of these instances as the bagLabel.
                    2.2 Set all other labels as 0
        '''
        assert currentY.ndim == 3
        assert k <= currentY.shape[1]
        assert k > 0
        # predicted label for each instance is max of softmax
        predictedLabels = np.argmax(softmaxOut, axis=2)
        scoreList = []
        # classScores[i] is a 2d array where a[j,k] is the longest
        # string of consecutive class labels i in bag j ending at instance k
        classScores = [-1]
        for i in range(1, numClasses):
            scores = self.__getLengthScores(predictedLabels, val=i)
            classScores.append(scores)
            length = np.max(scores, axis=1)
            scoreList.append(length)
        scoreList = np.array(scoreList)
        scoreList = scoreList.T
        # longestContinuousClass[i] is the class label having
        # longest substring in bag i
        longestContinuousClass = np.argmax(scoreList, axis=1) + 1
        # longestContinuousClassLength[i] is length of 
        # longest class substring in bag i
        longestContinuousClassLength = np.max(scoreList, axis=1)
        assert longestContinuousClass.ndim == 1
        assert longestContinuousClass.shape[0] == bagLabel.shape[0]
        assert longestContinuousClassLength.ndim == 1
        assert longestContinuousClassLength.shape[0] == bagLabel.shape[0]
        newY = np.array(currentY)
        index = (bagLabel != 0)
        indexList = np.where(index)[0]
        # iterate through all non-zero bags
        for i in indexList:
            # longest continuous class for this bag
            lcc = longestContinuousClass[i]
            # length of longest continuous class for this bag
            lccl = int(longestContinuousClassLength[i])
            # if bagLabel is not the same as longest continuous
            # class, don't update
            if lcc != bagLabel[i]:
                continue
            # we check for longest string to be at least k
            if lccl < k:
                continue
            lengths = classScores[lcc][i]
            assert np.max(lengths) == lccl
            possibleCandidates = np.where(lengths == lccl)[0]
            # stores (candidateIndex, sum of probabilities
            # over window for this index) pairs
            sumProbsAcrossLongest = {}
            for candidate in possibleCandidates:
                sumProbsAcrossLongest[candidate] = 0.0
                # sum the probabilities over the continuous substring
                for j in range(0, lccl):
                    sumProbsAcrossLongest[candidate] += softmaxOut[i, candidate-j, lcc]
            # we want only the one with maximum sum of
            # probabilities; sort dict by value
            sortedProbs = sorted(sumProbsAcrossLongest.items(),key=lambda x: x[1], reverse=True)
            bestCandidate = sortedProbs[0][0]
            # apart from (bestCanditate-lcc,bestCandidate] label
            # everything else as 0
            newY[i, :, :] = 0
            newY[i, :, 0] = 1
            newY[i, bestCandidate-lccl+1:bestCandidate+1, 0] = 0
            newY[i, bestCandidate-lccl+1:bestCandidate+1, lcc] = 1
        return newY

    def feedDictFunc(self, **kwargs):
        '''
        Construct feed dict from graph objects
        '''
        return None


Instructions for updating:
non-resource variables are not supported in the long term


In [12]:
# Copyright (c) Microsoft Corporation. All rights reserved.
# Licensed under the MIT license.

from __future__ import print_function
import tensorflow as tf
import numpy as np
import scipy.cluster
import scipy.spatial
import os


def medianHeuristic(data, projectionDimension, numPrototypes, W_init=None):
    '''
    This method can be used to estimate gamma for ProtoNN. An approximation to
    median heuristic is used here.
    1. First the data is collapsed into the projectionDimension by W_init. If
    W_init is not provided, it is initialized from a random normal(0, 1). Hence
    data normalization is essential.
    2. Prototype are computed by running a  k-means clustering on the projected
    data.
    3. The median distance is then estimated by calculating median distance
    between prototypes and projected data points.

    data needs to be [-1, numFeats]
    If using this method to initialize gamma, please use the W and B as well.

    TODO: Return estimate of Z (prototype labels) based on cluster centroids
    andand labels

    TODO: Clustering fails due to singularity error if projecting upwards

    W [dxd_cap]
    B [d_cap, m]
    returns gamma, W, B
    '''
    assert data.ndim == 2
    X = data
    featDim = data.shape[1]
    if projectionDimension > featDim:
        print("Warning: Projection dimension > feature dimension. Gamma")
        print("\t estimation due to median heuristic could fail.")
        print("\tTo retain the projection dataDimension, provide")
        print("\ta value for gamma.")

    if W_init is None:
        W_init = np.random.normal(size=[featDim, projectionDimension])
    W = W_init
    XW = np.matmul(X, W)
    assert XW.shape[1] == projectionDimension
    assert XW.shape[0] == len(X)
    # Requires [N x d_cap] data matrix of N observations of d_cap-dimension and
    # the number of centroids m. Returns, [n x d_cap] centroids and
    # elementwise center information.
    B, centers = scipy.cluster.vq.kmeans2(XW, numPrototypes)
    # Requires two matrices. Number of observations x dimension of observation
    # space. Distances[i,j] is the distance between XW[i] and B[j]
    distances = scipy.spatial.distance.cdist(XW, B, metric='euclidean')
    distances = np.reshape(distances, [-1])
    gamma = np.median(distances)
    gamma = 1 / (2.5 * gamma)
    return gamma.astype('float32'), W.astype('float32'), B.T.astype('float32')


def multiClassHingeLoss(logits, label, batch_th):
    '''
    MultiClassHingeLoss to match C++ Version - No TF internal version
    '''
    flatLogits = tf.reshape(logits, [-1, ])
    label_ = tf.argmax(label, 1)

    correctId = tf.range(0, batch_th) * label.shape[1] + label_
    correctLogit = tf.gather(flatLogits, correctId)

    maxLabel = tf.argmax(logits, 1)
    top2, _ = tf.nn.top_k(logits, k=2, sorted=True)

    wrongMaxLogit = tf.where(
        tf.equal(maxLabel, label_), top2[:, 1], top2[:, 0])

    return tf.reduce_mean(tf.nn.relu(1. + wrongMaxLogit - correctLogit))


def crossEntropyLoss(logits, label):
    '''
    Cross Entropy loss for MultiClass case in joint training for
    faster convergence
    '''
    return tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,
                                                   labels=tf.stop_gradient(label)))


def mean_absolute_error(logits, label):
    '''
    Function to compute the mean absolute error.
    '''
    return tf.reduce_mean(tf.abs(tf.subtract(logits, label)))


def hardThreshold(A, s):
    '''
    Hard thresholding function on Tensor A with sparsity s
    '''
    A_ = np.copy(A)
    A_ = A_.ravel()
    if len(A_) > 0:
        th = np.percentile(np.abs(A_), (1 - s) * 100.0, interpolation='higher')
        A_[np.abs(A_) < th] = 0.0
    A_ = A_.reshape(A.shape)
    return A_


def copySupport(src, dest):
    '''
    copy support of src tensor to dest tensor
    '''
    support = np.nonzero(src)
    dest_ = dest
    dest = np.zeros(dest_.shape)
    dest[support] = dest_[support]
    return dest


def countnnZ(A, s, bytesPerVar=4):
    '''
    Returns # of non-zeros and representative size of the tensor
    Uses dense for s >= 0.5 - 4 byte
    Else uses sparse - 8 byte
    '''
    params = 1
    hasSparse = False
    for i in range(0, len(A.shape)):
        params *= int(A.shape[i])
    if s < 0.5:
        nnZ = np.ceil(params * s)
        hasSparse = True
        return nnZ, nnZ * 2 * bytesPerVar, hasSparse
    else:
        nnZ = params
        return nnZ, nnZ * bytesPerVar, hasSparse


def getConfusionMatrix(predicted, target, numClasses):
    '''
    Returns a confusion matrix for a multiclass classification
    problem. `predicted` is a 1-D array of integers representing
    the predicted classes and `target` is the target classes.

    confusion[i][j]: Number of elements of class j
        predicted as class i
    Labels are assumed to be in range(0, numClasses)
    Use`printFormattedConfusionMatrix` to echo the confusion matrix
    in a user friendly form.
    '''
    assert(predicted.ndim == 1)
    assert(target.ndim == 1)
    arr = np.zeros([numClasses, numClasses])

    for i in range(len(predicted)):
        arr[predicted[i]][target[i]] += 1
    return arr


def printFormattedConfusionMatrix(matrix):
    '''
    Given a 2D confusion matrix, prints it in a human readable way.
    The confusion matrix is expected to be a 2D numpy array with
    square dimensions
    '''
    assert(matrix.ndim == 2)
    assert(matrix.shape[0] == matrix.shape[1])
    RECALL = 'Recall'
    PRECISION = 'PRECISION'
    print("|%s|" % ('True->'), end='')
    for i in range(matrix.shape[0]):
        print("%7d|" % i, end='')
    print("%s|" % 'Precision')

    print("|%s|" % ('-' * len(RECALL)), end='')
    for i in range(matrix.shape[0]):
        print("%s|" % ('-' * 7), end='')
    print("%s|" % ('-' * len(PRECISION)))

    precisionlist = np.sum(matrix, axis=1)
    recalllist = np.sum(matrix, axis=0)
    precisionlist = [matrix[i][i] / x if x !=
                     0 else -1 for i, x in enumerate(precisionlist)]
    recalllist = [matrix[i][i] / x if x !=
                  0 else -1 for i, x in enumerate(recalllist)]
    for i in range(matrix.shape[0]):
        # len recall = 6
        print("|%6d|" % (i), end='')
        for j in range(matrix.shape[0]):
            print("%7d|" % (matrix[i][j]), end='')
        print("%s" % (" " * (len(PRECISION) - 7)), end='')
        if precisionlist[i] != -1:
            print("%1.5f|" % precisionlist[i])
        else:
            print("%7s|" % "nan")

    print("|%s|" % ('-' * len(RECALL)), end='')
    for i in range(matrix.shape[0]):
        print("%s|" % ('-' * 7), end='')
    print("%s|" % ('-' * len(PRECISION)))
    print("|%s|" % ('Recall'), end='')

    for i in range(matrix.shape[0]):
        if recalllist[i] != -1:
            print("%1.5f|" % (recalllist[i]), end='')
        else:
            print("%7s|" % "nan", end='')

    print('%s|' % (' ' * len(PRECISION)))


def getPrecisionRecall(cmatrix, label=1):
    trueP = cmatrix[label][label]
    denom = np.sum(cmatrix, axis=0)[label]
    if denom == 0:
        denom = 1
    recall = trueP / denom
    denom = np.sum(cmatrix, axis=1)[label]
    if denom == 0:
        denom = 1
    precision = trueP / denom
    return precision, recall


def getMacroPrecisionRecall(cmatrix):
    # TP + FP
    precisionlist = np.sum(cmatrix, axis=1)
    # TP + FN
    recalllist = np.sum(cmatrix, axis=0)
    precisionlist__ = [cmatrix[i][i] / x if x !=
                       0 else 0 for i, x in enumerate(precisionlist)]
    recalllist__ = [cmatrix[i][i] / x if x !=
                    0 else 0 for i, x in enumerate(recalllist)]
    precision = np.sum(precisionlist__)
    precision /= len(precisionlist__)
    recall = np.sum(recalllist__)
    recall /= len(recalllist__)
    return precision, recall


def getMicroPrecisionRecall(cmatrix):
    # TP + FP
    precisionlist = np.sum(cmatrix, axis=1)
    # TP + FN
    recalllist = np.sum(cmatrix, axis=0)
    num = 0.0
    for i in range(len(cmatrix)):
        num += cmatrix[i][i]

    precision = num / np.sum(precisionlist)
    recall = num / np.sum(recalllist)
    return precision, recall


def getMacroMicroFScore(cmatrix):
    '''
    Returns macro and micro f-scores.
    Refer: http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.104.8244&rep=rep1&type=pdf
    '''
    precisionlist = np.sum(cmatrix, axis=1)
    recalllist = np.sum(cmatrix, axis=0)
    precisionlist__ = [cmatrix[i][i] / x if x !=
                       0 else 0 for i, x in enumerate(precisionlist)]
    recalllist__ = [cmatrix[i][i] / x if x !=
                    0 else 0 for i, x in enumerate(recalllist)]
    macro = 0.0
    for i in range(len(precisionlist)):
        denom = precisionlist__[i] + recalllist__[i]
        numer = precisionlist__[i] * recalllist__[i] * 2
        if denom == 0:
            denom = 1
        macro += numer / denom
    macro /= len(precisionlist)

    num = 0.0
    for i in range(len(precisionlist)):
        num += cmatrix[i][i]

    denom1 = np.sum(precisionlist)
    denom2 = np.sum(recalllist)
    pi = num / denom1
    rho = num / denom2
    denom = pi + rho
    if denom == 0:
        denom = 1
    micro = 2 * pi * rho / denom
    return macro, micro


def restructreMatrixBonsaiSeeDot(A, nClasses, nNodes):
    '''
    Restructures a matrix from [nNodes*nClasses, Proj] to 
    [nClasses*nNodes, Proj] for SeeDot
    '''
    tempMatrix = np.zeros(A.shape)
    rowIndex = 0

    for i in range(0, nClasses):
        for j in range(0, nNodes):
            tempMatrix[rowIndex] = A[j * nClasses + i]
            rowIndex += 1

    return tempMatrix


class GraphManager:
    '''
    Manages saving and restoring graphs. Designed to be used with EMI-RNN
    though is general enough to be useful otherwise as well.
    '''

    def __init__(self):
        pass

    def checkpointModel(self, saver, sess, modelPrefix,
                        globalStep=1000, redirFile=None):
        saver.save(sess, modelPrefix, global_step=globalStep)
        print('Model saved to %s, global_step %d' % (modelPrefix, globalStep),
              file=redirFile)

    def loadCheckpoint(self, sess, modelPrefix, globalStep,
                       redirFile=None):
        metaname = modelPrefix + '-%d.meta' % globalStep
        basename = os.path.basename(metaname)
        fileList = os.listdir(os.path.dirname(modelPrefix))
        fileList = [x for x in fileList if x.startswith(basename)]
        assert len(fileList) > 0, 'Checkpoint file not found'
        msg = 'Too many or too few checkpoint files for globalStep: %d' % globalStep
        assert len(fileList) == 1, msg
        chkpt = basename + '/' + fileList[0]
        saver = tf.train.import_meta_graph(metaname)
        metaname = metaname[:-5]
        saver.restore(sess, metaname)
        graph = tf.get_default_graph()
        return graph


# Computation Graph

In [13]:
# Define the linear secondary classifier
def createExtendedGraph(self, baseOutput, *args, **kwargs):
    W1 = tf.Variable(np.random.normal(size=[NUM_HIDDEN, NUM_OUTPUT]).astype('float32'), name='W1')
    B1 = tf.Variable(np.random.normal(size=[NUM_OUTPUT]).astype('float32'), name='B1')
    y_cap = tf.add(tf.tensordot(baseOutput, W1, axes=1), B1, name='y_cap_tata')
    self.output = y_cap
    self.graphCreated = True

def restoreExtendedGraph(self, graph, *args, **kwargs):
    y_cap = graph.get_tensor_by_name('y_cap_tata:0')
    self.output = y_cap
    self.graphCreated = True
    
def feedDictFunc(self, keep_prob=None, inference=False, **kwargs):
    if inference is False:
        feedDict = {self._emiGraph.keep_prob: keep_prob}
    else:
        feedDict = {self._emiGraph.keep_prob: 1.0}
    return feedDict

    
EMI_FastGRNN._createExtendedGraph = createExtendedGraph
EMI_FastGRNN._restoreExtendedGraph = restoreExtendedGraph
if USE_DROPOUT is True:
    EMI_FastGRNN.feedDictFunc = feedDictFunc

In [14]:
inputPipeline = EMI_DataPipeline(NUM_SUBINSTANCE, NUM_TIMESTEPS, NUM_FEATS, NUM_OUTPUT)
emiFastGRNN = EMI_FastGRNN(NUM_SUBINSTANCE, NUM_HIDDEN, NUM_TIMESTEPS, NUM_FEATS, wRank=WRANK, uRank=URANK, 
                           gate_non_linearity=GATE_NL, update_non_linearity=UPDATE_NL, useDropout=USE_DROPOUT)
emiTrainer = EMI_Trainer(NUM_TIMESTEPS, NUM_OUTPUT, lossType='xentropy')

In [15]:
print("x_train shape is:", x_train.shape)
print("y_train shape is:", y_train.shape)
print("x_test shape is:", x_val.shape)
print("y_test shape is:", y_val.shape)

x_train shape is: (6108, 4, 88, 8)
y_train shape is: (6108, 4, 3)
x_test shape is: (679, 4, 88, 8)
y_test shape is: (679, 4, 3)


In [16]:
#tf.reset_default_graph()
g1 = tf.Graph()    
with g1.as_default():
    # Obtain the iterators to each batch of the data
    x_batch, y_batch = inputPipeline()
    # Create the forward computation graph based on the iterators
    y_cap = emiFastGRNN(x_batch)
    # Create loss graphs and training routines
    emiTrainer(y_cap, y_batch)

Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


AttributeError: 'EMI_Trainer' object has no attribute '_EMI_Trainer__createTrainOp'

# EMI Driver

In [88]:
with g1.as_default():
    emiDriver = EMI_Driver(inputPipeline, emiFastGRNN, emiTrainer)

emiDriver.initializeSession(g1)
y_updated, modelStats = emiDriver.run(numClasses=NUM_OUTPUT, x_train=x_train,
                                      y_train=y_train, bag_train=BAG_TRAIN,
                                      x_val=x_val, y_val=y_val, bag_val=BAG_VAL,
                                      numIter=NUM_ITER, keep_prob=KEEP_PROB,
                                      numRounds=NUM_ROUNDS, batchSize=BATCH_SIZE,
                                      numEpochs=NUM_EPOCHS, modelPrefix=MODEL_PREFIX,
                                      fracEMI=0.5, updatePolicy='top-k', k=1)

AssertionError: Have you invoked __call__()

In [16]:
# Early Prediction Policy: We make an early prediction based on the predicted classes
#     probability. If the predicted class probability > minProb at some step, we make
#     a prediction at that step.
def earlyPolicy_minProb(instanceOut, minProb, **kwargs):
    assert instanceOut.ndim == 2
    classes = np.argmax(instanceOut, axis=1)
    prob = np.max(instanceOut, axis=1)
    index = np.where(prob >= minProb)[0]
    if len(index) == 0:
        assert (len(instanceOut) - 1) == (len(classes) - 1)
        return classes[-1], len(instanceOut) - 1
    index = index[0]
    return classes[index], index

def getEarlySaving(predictionStep, numTimeSteps, returnTotal=False):
    predictionStep = predictionStep + 1
    predictionStep = np.reshape(predictionStep, -1)
    totalSteps = np.sum(predictionStep)
    maxSteps = len(predictionStep) * numTimeSteps
    savings = 1.0 - (totalSteps / maxSteps)
    if returnTotal:
        return savings, totalSteps
    return savings

In [17]:
k = 2
predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb, minProb=0.99)
bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
print('Accuracy at k = %d: %f' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))))
print('Additional savings: %f' % getEarlySaving(predictionStep, NUM_TIMESTEPS))

Accuracy at k = 2: 0.998567
Additional savings: 0.960761


In [18]:
# A slightly more detailed analysis method is provided. 
df = emiDriver.analyseModel(predictions, BAG_TEST, NUM_SUBINSTANCE, NUM_OUTPUT)

   len       acc  macro-fsc  macro-pre  macro-rec  micro-fsc  micro-pre  \
0    1  0.998831   0.998532   0.998504   0.998561   0.998831   0.998831   
1    2  0.998567   0.998295   0.998517   0.998074   0.998567   0.998567   
2    3  0.997850   0.997675   0.998287   0.997069   0.997850   0.997850   
3    4  0.996040   0.995706   0.997310   0.994133   0.996040   0.996040   

   micro-rec  
0   0.998831  
1   0.998567  
2   0.997850  
3   0.996040  
Max accuracy 0.998831 at subsequencelength 1
Max micro-f 0.998831 at subsequencelength 1
Micro-precision 0.998831 at subsequencelength 1
Micro-recall 0.998831 at subsequencelength 1
Max macro-f 0.998532 at subsequencelength 1
macro-precision 0.998504 at subsequencelength 1
macro-recall 0.998561 at subsequencelength 1


## Picking the best model

In [19]:
devnull = open(os.devnull, 'r')
for val in modelStats:
    round_, acc, modelPrefix, globalStep = val
    emiDriver.loadSavedGraphToNewSession(modelPrefix, globalStep, redirFile=devnull)
    predictions, predictionStep = emiDriver.getInstancePredictions(x_test, y_test, earlyPolicy_minProb,
                                                               minProb=0.99, keep_prob=1.0)
 
    bagPredictions = emiDriver.getBagPredictions(predictions, minSubsequenceLen=k, numClass=NUM_OUTPUT)
    print("Round: %2d, Validation accuracy: %.4f" % (round_, acc), end='')
    print(', Test Accuracy (k = %d): %f, ' % (k,  np.mean((bagPredictions == BAG_TEST).astype(int))), end='')
    print('Additional savings: %f' % getEarlySaving(predictionStep, NUM_TIMESTEPS)) 

INFO:tensorflow:Restoring parameters from /home/sf/data/WESAD/Fast_GRNN/88_30/models/model-fgrnn-1003
Round:  0, Validation accuracy: 0.9927, Test Accuracy (k = 2): 0.960361, Additional savings: 0.372858
INFO:tensorflow:Restoring parameters from /home/sf/data/WESAD/Fast_GRNN/88_30/models/model-fgrnn-1007
Round:  1, Validation accuracy: 0.9973, Test Accuracy (k = 2): 0.926303, Additional savings: 0.508829
INFO:tensorflow:Restoring parameters from /home/sf/data/WESAD/Fast_GRNN/88_30/models/model-fgrnn-1011
Round:  2, Validation accuracy: 0.9964, Test Accuracy (k = 2): 0.950743, Additional savings: 0.585428
INFO:tensorflow:Restoring parameters from /home/sf/data/WESAD/Fast_GRNN/88_30/models/model-fgrnn-1015
Round:  3, Validation accuracy: 0.9972, Test Accuracy (k = 2): 0.997435, Additional savings: 0.945828
INFO:tensorflow:Restoring parameters from /home/sf/data/WESAD/Fast_GRNN/88_30/models/model-fgrnn-1018
Round:  4, Validation accuracy: 0.9985, Test Accuracy (k = 2): 0.998491, Additiona

In [20]:
dataset="WESAD"
model="fast-grnn"
params = {
    "NUM_HIDDEN" : 128,
    "NUM_TIMESTEPS" : 700, #subinstance length.
    "NUM_FEATS" : 8,
    "FORGET_BIAS" : 1.0,
    "NUM_OUTPUT" : 3,
    "USE_DROPOUT" : 0, # '1' -> True. '0' -> False
    "KEEP_PROB" : 0.9,
    "UPDATE_NL" : "quantTanh",
    "GATE_NL" : "quantSigm",
    "WRANK" : 5,
    "URANK" : 6,
    "PREFETCH_NUM" : 5,
    "BATCH_SIZE" : 175,
    "NUM_EPOCHS" : 3,
    "NUM_ITER" : 4,
    "NUM_ROUNDS" : 4,
    "MODEL_PREFIX" : dataset + '/model-' + str(model)
}

fast_dict = {**params}
fast_dict["k"] = k
fast_dict["accuracy"] = np.mean((bagPredictions == BAG_TEST).astype(int))
fast_dict["additional_savings"] = getEarlySaving(predictionStep, NUM_TIMESTEPS)
fast_dict["y_test"] = BAG_TEST
fast_dict["y_pred"] = bagPredictions

In [21]:
# A slightly more detailed analysis method is provided. 
df = emiDriver.analyseModel(predictions, BAG_TEST, NUM_SUBINSTANCE, NUM_OUTPUT)
print (tabulate(df, headers=list(df.columns), tablefmt='grid'))

dirname = "/home/sf/data/WESAD/Fast_GRNN/"
pathlib.Path(dirname).mkdir(parents=True, exist_ok=True)
print ("Results for this run have been saved at" , dirname, ".")

now = datetime.datetime.now()
filename = list((str(now.year),"-",str(now.month),"-",str(now.day),"|",str(now.hour),"-",str(now.minute)))
filename = ''.join(filename)

#Save the dictionary containing the params and the results.
pkl.dump(fast_dict,open(dirname + "/fast_dict_" + filename + ".pkl",mode='wb'))

   len       acc  macro-fsc  macro-pre  macro-rec  micro-fsc  micro-pre  \
0    1  0.998831   0.998532   0.998504   0.998561   0.998831   0.998831   
1    2  0.998567   0.998295   0.998517   0.998074   0.998567   0.998567   
2    3  0.997850   0.997675   0.998287   0.997069   0.997850   0.997850   
3    4  0.996040   0.995706   0.997310   0.994133   0.996040   0.996040   

   micro-rec  
0   0.998831  
1   0.998567  
2   0.997850  
3   0.996040  
Max accuracy 0.998831 at subsequencelength 1
Max micro-f 0.998831 at subsequencelength 1
Micro-precision 0.998831 at subsequencelength 1
Micro-recall 0.998831 at subsequencelength 1
Max macro-f 0.998532 at subsequencelength 1
macro-precision 0.998504 at subsequencelength 1
macro-recall 0.998561 at subsequencelength 1
+----+-------+----------+-------------+-------------+-------------+-------------+-------------+-------------+
|    |   len |      acc |   macro-fsc |   macro-pre |   macro-rec |   micro-fsc |   micro-pre |   micro-rec |
+====+====